In [1]:
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import json
from pandas import json_normalize
import locale
import time
#import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import ttest_rel

import locale
locale.setlocale(locale.LC_ALL, 'de_DE')

%store -r columnNames

#pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
# pd.set_option('display.max_colwidth', None)  # or 199

# pd.describe_option('display')
#plt.close("all")

In [2]:
# Create color set bades on UR CD colors
urCdColors = [
    "rgb(156,0,75)", # HEIDENELKENROT
    "rgb(0,155,119)", # TÜRKISGRÜN
    "rgb(0,135,178)", # SPEKTRALBLAU
    "rgb(174,167,0)", # LÄRCHENNADELGRÜN
    "rgb(236,98,0)", # ORANGEROT
    "rgb(191,0,42)", # HEUCHERAROT
    "rgb(79,184,0)", # BLATTGRÜN
    "rgb(0,137,147)", # EISVOGELBLAU
    "rgb(0,85,106)", # CAPRIBLAU
    "rgb(205,211,15)", # URANGELB
    "rgb(236,188,0)" # VATIKANGELB
]

# Go table header colors
headerColor = 'grey'
rowEvenColor = 'lightgrey'
rowOddColor = 'white'

# Get Results

In [3]:
surveyData=pd.read_csv('justCompletedAttempts.csv', sep=';')
print("Anzahl an kompletten Daten:", len(surveyData))

# Convert to seconds
def secondsToTime(seconds): 
    timeString =  str( time.strftime('%H:%M:%S', time.gmtime(seconds)) ) + str( seconds )[-3:]
    return timeString

# Converts mood value in float
def moodToFloatVal(moodString):
    moodFloat = 0.0

    if moodString == "Schlecht":
        moodFloat = -2.0
    elif moodString == "Nicht gut":
        moodFloat = -1.0
    elif moodString == "Neutral":
        moodFloat = 0.0
    elif moodString == "Gut":
        moodFloat = 1.0
    elif moodString == "Sehr gut":
        moodFloat = 2.0
        
    return moodFloat

# Add extra fields for comparison
surveyData['duration-over-all-time'] = surveyData.apply(lambda row: secondsToTime(row['duration-over-all']), axis=1)
surveyData['duration-date-based'] = surveyData.apply(lambda row: pd.to_datetime(row['date-last'])-pd.to_datetime(row['date-start']), axis=1)
surveyData['duration-date-based-in-seconds'] = surveyData.apply(lambda row: row['duration-date-based'].total_seconds(), axis=1)
surveyData['duration-delta-all-vs-start-end'] = surveyData.apply(lambda row: row['duration-over-all']-row['duration-date-based-in-seconds'], axis=1)
surveyData['duration-shortest-val'] = surveyData.apply(lambda row: min(row['duration-over-all'], row['duration-date-based-in-seconds']), axis=1)
surveyData['duration-shortest-val-time'] = surveyData.apply(lambda row: secondsToTime(row['duration-shortest-val']), axis=1)
surveyData['mood-start-val'] = surveyData.apply(lambda row: moodToFloatVal(row['mood-start']), axis=1)
surveyData['mood-end-val'] = surveyData.apply(lambda row: moodToFloatVal(row['mood-end']), axis=1)
surveyData['mood-delta'] = surveyData.apply(lambda row: row['mood-end-val']-row['mood-start-val'], axis=1)

# Build groups depending on what viz participant had to solve first

listFirstParticipants=surveyData[surveyData['list-question-number']==6]
chordFirstParticipants=surveyData[surveyData['chord-question-number']==6]
mapFirstParticipants=surveyData[surveyData['map-question-number']==6]

Anzahl an kompletten Daten: 203


## Socio-Demographic Data
### Age

In [4]:
df = surveyData[['age', 'highest-preferenced-viz']]

# Label strings
chartLabel = f"Altersverteilung,pythi n={len(df)}"
categoryGroups = ["Alter gesamt", "nach Listen Präferenz", "nach Chord Präferenz", "nach Map Präferenz"]
yLabel = "Alter in Jahren"
fig = go.Figure()

# Age over all
fig.add_trace(go.Box(y=df['age'], name=categoryGroups[0],
                marker_color = urCdColors[0]))
# Age preferred List
fig.add_trace(go.Box(y=df[df['highest-preferenced-viz'] == "Liste"]['age'], name=categoryGroups[1],
                marker_color = urCdColors[1]))
# Age preferred Chord
fig.add_trace(go.Box(y=df[df['highest-preferenced-viz'] == "Chord"]['age'], name=categoryGroups[2],
                marker_color = urCdColors[2]))
# Age preferred Map
fig.add_trace(go.Box(y=df[df['highest-preferenced-viz'] == "Map"]['age'], name=categoryGroups[3],
                marker_color = urCdColors[3]))
fig.update_layout(
    title=chartLabel,
    xaxis_title="Alter nach Präferenz",
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.update_layout(title_text=chartLabel)
fig.show()

# Table age
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Kennwert</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>", f"<b>{categoryGroups[3]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['<b>Maximum</b>', '<b>Oberes Quartil</b>', '<b>Median</b>', '<b>Unteres Quartil</b>', '<b>Minimum</b>'],
      [42, 32, 22, 20, 18],
      [33, 25.5, 22, 20, 18],
      [57, 36, 24, 21, 18],
      [43, 30, 22, 20, 18]],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

### Gender

In [5]:
df = surveyData[['sex']]

df.fillna('nicht angegeben', inplace=True)
df = df.groupby(['sex'], dropna=False)['sex'].count().reset_index(name='count')

# Label strings
chartLabel = f"Verteiltung nach Geschlecht, n={len(surveyData[['sex']])}"
categoryGroups = ["Männlich", "Keine Angabe", "Weiblich"]

fig = go.Figure(data=[go.Pie( labels=categoryGroups, values=df['count'])])
fig.update_traces(textinfo='percent+label', marker=dict(colors=urCdColors))
fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    width=600,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)
fig.show()

# Table sex
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Geschlecht</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>", "<b>Gesamt</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      # [surveyData[timeCleanedData['sex'] == 'männlich'], surveyData[surveyData['sex'] == 'nicht angegeben'], surveyData[surveyData['sex'] == 'weiblich']]],
        ['<b>Anzahl</b>'],
        [len(surveyData[surveyData['sex'] == 'männlich'])],
        [surveyData['sex'].isnull().sum()],
        [len(surveyData[surveyData['sex'] == 'weiblich'])],
        [len(surveyData[['sex']])]
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

/var/folders/s3/0k49_ks152z51hj36nht1qyr0000gn/T/ipykernel_8852/3112003316.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Education

In [6]:
df = surveyData[['education']]
df = df.groupby(['education'])['education'].count().reset_index(name='count')

# Label strings
chartLabel = f"Verteilung nach Ausbildung, n={len(surveyData[['education']])}"
categoryGroups = ["Qualifizierender Hauptschulabschluss", "Mittlere Reife", "Hochschulreife", "Studium", "Promotion"]
yLabel = "Anzahl"
xLabel = "Höchster Bildungsabschluss"

data = [
    df.loc[df['education'] == categoryGroups[0], 'count'].iloc[0],
    df.loc[df['education'] == categoryGroups[1], 'count'].iloc[0],
    df.loc[df['education'] == categoryGroups[2], 'count'].iloc[0],
    df.loc[df['education'] == categoryGroups[3], 'count'].iloc[0],
    df.loc[df['education'] == categoryGroups[4], 'count'].iloc[0],
] 

# print(df[['education'] == categoryGroups[0]].count)
print(df.loc[df['education'] == categoryGroups[0], 'count'].iloc[0])

fig = px.bar(df, x=categoryGroups, y=data, text_auto='.0s',
            labels={'x':'Grad des Bildungsabschluss', 'y':'Anzahl'},
            title=chartLabel,
            )
fig.update_traces(marker_color=urCdColors, opacity=0.75)
# Print Optimization
fig.update_layout(width=600, height=600);
fig.show()

# Table
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Bildungsgrad nach Abschluss</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>", f"<b>{categoryGroups[3]}</b>", f"<b>{categoryGroups[4]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      # [surveyData[surveyData['sex'] == 'männlich'], surveyData[surveyData['sex'] == 'nicht angegeben'], surveyData[surveyData['sex'] == 'weiblich']]],
        ['<b>Anzahl</b>'],
        [data[0]],
        [data[1]],
        [data[2]],
        [data[3]],
        [data[4]]
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

4


### Profession

In [7]:
df = surveyData[['profession']]

# print(df)

# Replacements
pd.options.mode.chained_assignment = None  # default='warn'
df.loc[df['profession'].str.contains('student', case=False), 'profession'] = 'Student:in'
df.loc[df['profession'].str.contains('Studium', case=False), 'profession'] = 'Student:in'
df.loc[df['profession'].str.contains('Studierend', case=False), 'profession'] = 'Student:in'
df.loc[df['profession'].str.contains('Medientechnik und Produktion', case=False), 'profession'] = 'Student:in'
df.loc[df['profession'].str.contains('Barkeeper', case=False), 'profession'] = 'Barkeeper:in'
df.loc[df['profession'].str.contains('ingenieur', case=False), 'profession'] = 'Ingenieur:in'
df.loc[df['profession'].str.contains('Dipl.Ing.grad', case=False), 'profession'] = 'Ingenieur:in'
df.loc[df['profession'].str.contains('Software', case=False), 'profession'] = 'Software Entwickler:in'
df.loc[df['profession'].str.contains('Mediengestalt', case=False), 'profession'] = 'Mediengestalter:in'
df.loc[df['profession'].str.contains('Ehrenamt', case=False), 'profession'] = 'Arbeitslos/Ehrenamt'
df.loc[df['profession'].str.contains('Arbeitslos', case=False), 'profession'] = 'Arbeitslos/Ehrenamt'
df.loc[df['profession'].str.contains('-', case=False), 'profession'] = 'Arbeitslos/Ehrenamt'
df.loc[df['profession'].str.contains('/', case=False), 'profession'] = 'Arbeitslos/Ehrenamt'
df.loc[df['profession'].str.contains('Keinen', case=False), 'profession'] = 'Arbeitslos/Ehrenamt'
df.loc[df['profession'].str.contains('Nicht vorhanden', case=False), 'profession'] = 'Arbeitslos/Ehrenamt'
df.loc[df['profession'].str.contains('Beamt', case=False), 'profession'] = 'Beamtin/Beamter'
df.loc[df['profession'].str.contains('rzt', case=False), 'profession'] = 'Ärtzin/Arzt'
df.loc[df['profession'].str.contains('Kieferorthopädin', case=False), 'profession'] = 'Ärtzin/Arzt'
df.loc[df['profession'].str.contains('Künstler', case=False), 'profession'] = 'Künstler:in'
df.loc[df['profession'].str.contains('Bäderbetriebe', case=False), 'profession'] = 'Bademeister:in'
df.loc[df['profession'].str.contains('Bademeister:in', case=False), 'profession'] = 'Fachangestellte:r für Bäderbetriebe'
df.loc[df['profession'].str.contains('^Angestellte', case=False), 'profession'] = 'Angestellte/Angestellter'
df.loc[df['profession'].str.contains('Systemadministrator', case=False), 'profession'] = 'IT Systemadministrator:in'
df.loc[df['profession'].str.contains('senschaftlicher Mitarb', case=False), 'profession'] = 'Wiss. Mitarbeiter:in'
df.loc[df['profession'].str.contains('senschaftliche Mitarb', case=False), 'profession'] = 'Wiss. Mitarbeiter:in'
df.loc[df['profession'].str.contains('wiissenschaftliche', case=False), 'profession'] = 'Wiss. Mitarbeiter:in'
df.loc[df['profession'].str.contains('wiss. Mitarb', case=False), 'profession'] = 'Wiss. Mitarbeiter:in'
df.loc[df['profession'].str.contains('Bankfachwirtin', case=False), 'profession'] = 'Bankfachwirt:in'
df.loc[df['profession'].str.contains('Beratung', case=False), 'profession'] = 'Berater:in'
df.loc[df['profession'].str.contains('Bibliothekar', case=False), 'profession'] = 'Bibliothekar:in'
df.loc[df['profession'].str.contains('Briefzusteller', case=False), 'profession'] = 'Briefzusteller:in/Postbot:in'
df.loc[df['profession'].str.contains('postbot', case=False), 'profession'] = 'Briefzusteller:in/Postbot:in'
df.loc[df['profession'].str.contains('Buchhändler', case=False), 'profession'] = 'Buchhändler:in'
df.loc[df['profession'].str.contains('Doktorand', case=False), 'profession'] = 'Doktorand:in'
df.loc[df['profession'].str.contains('kauffrau', case=False), 'profession'] = 'Kauffrau/Kaufmann'
df.loc[df['profession'].str.contains('Fachverkäuferin im Lebensmittelhandwerk', case=False), 'profession'] = 'Fachverkäufer:in im Lebensmittelhandwerk'
df.loc[df['profession'].str.contains('Fremdsprachenkorrespondentin', case=False), 'profession'] = 'Fremdsprachenkorrespondent:in'
df.loc[df['profession'].str.contains('Informatiker', case=False), 'profession'] = 'Informatiker:in'
df.loc[df['profession'].str.contains('Kassiererin', case=False), 'profession'] = 'Kassierer:in'
df.loc[df['profession'].str.contains('Landwirt', case=False), 'profession'] = 'Landwirt:in'
df.loc[df['profession'].str.contains('Manager', case=False), 'profession'] = 'Manager:in'
df.loc[df['profession'].str.contains('Lehrer', case=False), 'profession'] = 'Lehrer:in'
df.loc[df['profession'].str.contains('Mechaniker', case=False), 'profession'] = 'Mechaniker:in/Mechatroniker:in'
df.loc[df['profession'].str.contains('Mechatroniker', case=False), 'profession'] = 'Mechaniker:in/Mechatroniker:in'
df.loc[df['profession'].str.contains('Lektorin', case=False), 'profession'] = 'Lektor:in'
df.loc[df['profession'].str.contains('Musikvermittlerin', case=False), 'profession'] = 'Musikvermittler:in'
df.loc[df['profession'].str.contains('Professor', case=False), 'profession'] = 'Professor:in'
df.loc[df['profession'].str.contains('Projektassistenz', case=False), 'profession'] = 'Projektassistent:in'
df.loc[df['profession'].str.contains('Referent', case=False), 'profession'] = 'Referent:in'
df.loc[df['profession'].str.contains('Sachbearbeiterin', case=False), 'profession'] = 'Sachbearbeiter:in'
df.loc[df['profession'].str.contains('Schriftsteller', case=False), 'profession'] = 'Schriftsteller:in und Literaturwissenschaftler:in'
df.loc[df['profession'].str.contains('Sekretärin', case=False), 'profession'] = 'Sekretär:in'
df.loc[df['profession'].str.contains('Verwaltungsmitarbeiter', case=False), 'profession'] = 'Verwaltungsmitarbeiter:in'
df.loc[df['profession'].str.contains('Verwaltungsangestellte', case=False), 'profession'] = 'Verwaltungsmitarbeiter:in'
df.loc[df['profession'].str.contains('Vertriebsaußendienst', case=False), 'profession'] = 'Vertriebsaußendienstler:in'
df.loc[df['profession'].str.contains('Selbstständiger Fotograf', case=False), 'profession'] = 'Fotograf:in'
pd.options.mode.chained_assignment = 'warn'  # default='warn'

unifiedProfessionsCounts = df.groupby(['profession'])['profession'].count().reset_index(name='count')
sortedProfessions = unifiedProfessionsCounts.sort_values(['count', 'profession'], ascending=[False, True])

# Label strings
chartLabel = f"Verteilung nach Berufen, n={len(surveyData[['education']])}"
categoryGroups = ["Beruf", "Anzahl"]
xLabel = "Eigenschaft"
yLabel = "Beruf"

# Table
fig = go.Figure(data=[go.Table(
  header=dict(
    values=[f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[sortedProfessions.profession, sortedProfessions['count']],
  #     # [surveyData[surveyData['sex'] == 'männlich'], surveyData[surveyData['sex'] == 'nicht angegeben'], surveyData[surveyData['sex'] == 'weiblich']]],
  #       ['<b>Anzahl</b>'],
  #       [data[0]],
  #       [data[1]],
  #       [data[2]],
  #       [data[3]],
  #       [data[4]]
  #     ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor]*21],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    # yaxis_title=yLabel,
    # legend_title="Legend Title",
    height=1100,
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()
# sortedProfessions.to_csv("beruf.csv", encoding='utf-8', sep=';');



### Standard Deviation of Test Time

In [8]:
# Standard deviation time and mood
# df = surveyData[['duration-shortest-val', 'mood-delta']]
df = surveyData.loc[surveyData['duration-shortest-val'] < 6000][['duration-shortest-val', 'mood-delta']]

# Label strings
chartLabel = f"Zeitverteilung, n={len(df)} von 203, t < 100 Minuten"
categoryGroups = ["alle Teilnehmer:innen", "stabile Gefühlsentwicklung", "Verbesserung d. Gefühlsentwicklung", "Verschlechterung d. Gefühlsentwicklung"]
yLabel = "Dauer des Tests in Sekunden"
fig = go.Figure()

# Age over all
fig.add_trace(go.Box(y=df['duration-shortest-val'], name=categoryGroups[0],
                marker_color = urCdColors[0]))
# Age preferred List
fig.add_trace(go.Box(y=df[df['mood-delta'] == 0]['duration-shortest-val'], name=categoryGroups[1],
                marker_color = urCdColors[1]))
# Age preferred Chord
fig.add_trace(go.Box(y=df[df['mood-delta'] >= 1]['duration-shortest-val'], name=categoryGroups[2],
                marker_color = urCdColors[2]))
# Age preferred Map
fig.add_trace(go.Box(y=df[df['mood-delta'] <= -1]['duration-shortest-val'], name=categoryGroups[3],
                marker_color = urCdColors[3]))
fig.update_layout(
    title=chartLabel,
    xaxis_title="Zeitverteilung nach Gefühlsentwicklung",
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.update_layout(title_text=chartLabel)
fig.show()

# Table age
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Kennwert</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>", f"<b>{categoryGroups[3]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['<b>Maximum</b>', '<b>Oberes Quartil</b>', '<b>Median</b>', '<b>Unteres Quartil</b>', '<b>Minimum</b>'],
      # TODO: GET DIRECT VALUES
      [4367, 2505, 2015, 1253, 319],
      [4083,2494, 1983, 1272, 424],
      [3682, 2919, 2213, 1669, 319],
      [4414, 2350, 1312, 818, 393]],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()


## Mood Development

In [9]:
df = surveyData[['duration-shortest-val', 'mood-delta']]

df = df.groupby(['mood-delta'], dropna=False)['mood-delta'].count().reset_index(name='count')

# Label strings
chartLabel = f"Gefühlsentwicklung der Teilnehmer:innen, n={len(surveyData[['mood-delta']])}"
categoryGroups = ["Verschlechterung um 3 Grade", "Verschlechterung um 2 Grade", "Verschlechterung um 1 Grad", "Stabile Entwicklung", "Verbesserung um 1 Grad", "Verbesserung um 2 Grade"]

fig = go.Figure(data=[go.Pie( labels=categoryGroups, values=df['count'], sort=False, direction='clockwise')])
fig.update_traces(textinfo='value+percent+label', marker=dict(colors=urCdColors))
fig.update_layout(
    title={
      'text': chartLabel,
      'y':0.025,
      # 'x':0,
      'xanchor': 'left',
      'yanchor': 'top',
      'font_size': 24,
    },
    # yaxis_title=yLabel,
    width=1280,
    height=1080,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=24,
        # color="RebeccaPurple"
    )
)
fig.show()

# Table mood development
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Gefühlsentwicklung</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>", f"<b>{categoryGroups[3]}</b>", f"<b>{categoryGroups[4]}</b>", f"<b>{categoryGroups[5]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      # [surveyData[surveyData['sex'] == 'männlich'], surveyData[surveyData['sex'] == 'nicht angegeben'], surveyData[surveyData['sex'] == 'weiblich']]],
        ['<b>Anzahl</b>'],
        [df.iloc[0]['count']],
        [df.iloc[1]['count']],
        [df.iloc[2]['count']],
        [df.iloc[3]['count']],
        [df.iloc[4]['count']],
        [df.iloc[5]['count']]
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    # yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

### Mood Development To Choice

In [10]:
dfAll = surveyData[['duration-shortest-val', 'mood-delta', 'highest-preferenced-viz']]
dfAll = dfAll.groupby(['mood-delta'], dropna=False)['mood-delta'].count().reset_index(name='count')

dfListe = surveyData.loc[surveyData['highest-preferenced-viz'] == 'Liste'][['duration-shortest-val', 'mood-delta', 'highest-preferenced-viz']]
dfListe = dfListe.groupby(['mood-delta'], dropna=False)['mood-delta'].count().reset_index(name='count')

dfChord = surveyData.loc[surveyData['highest-preferenced-viz'] == 'Chord'][['duration-shortest-val', 'mood-delta', 'highest-preferenced-viz']]
dfChord = dfChord.groupby(['mood-delta'], dropna=False)['mood-delta'].count().reset_index(name='count')

dfMap = surveyData.loc[surveyData['highest-preferenced-viz'] == 'Map'][['duration-shortest-val', 'mood-delta', 'highest-preferenced-viz']]
dfMap = dfMap.groupby(['mood-delta'], dropna=False)['mood-delta'].count().reset_index(name='count')

# Label strings
chartLabel = f"Gefühlsentwicklung der Teilnehmer:innen, n={len(surveyData[['mood-delta']])}"
categoryGroups = ["Verschlechterung um 3 Grade", "Verschlechterung um 2 Grade", "Verschlechterung um 1 Grad", "Stabile Entwicklung", "Verbesserung um 1 Grad", "Verbesserung um 2 Grade"]

fig = make_subplots(rows=2, cols=3, 
    specs=[
        [None, {'type':'domain'}, None],
        [{'type':'domain'}, {'type':'domain'}, {'type':'domain'}],
    ],
    subplot_titles=("Gesamt","Liste favorisiert", "Chord  favorisiert",  "Map favorisiert"))

fig.add_trace(go.Pie(labels=categoryGroups, values=dfAll['count'], sort=False, direction='clockwise', marker_colors=urCdColors),
              1, 2)
fig.add_trace(go.Pie(labels=categoryGroups, values=dfListe['count'], sort=False, direction='clockwise', marker_colors=urCdColors),
              2, 1)
fig.add_trace(go.Pie(labels=[categoryGroups[0], categoryGroups[2],categoryGroups[3], categoryGroups[4], categoryGroups[5]], values=dfChord['count'], sort=False, direction='clockwise', marker_colors=[urCdColors[0], urCdColors[2], urCdColors[3], urCdColors[4], urCdColors[5]]),
              2, 2)
fig.add_trace(go.Pie(labels=[categoryGroups[2], categoryGroups[3], categoryGroups[4], categoryGroups[5]], values=dfMap['count'], sort=False, direction='clockwise', marker_colors=[urCdColors[2], urCdColors[3], urCdColors[4], urCdColors[5]]),
              2, 3)

fig.update_traces(textinfo='percent+label')
fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    height=850,
    #legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)
fig.update(layout_showlegend=True)
fig.update_annotations(yshift=70)
fig.show()


# Table mood development
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Gefühlsentwicklung</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>", f"<b>{categoryGroups[3]}</b>", f"<b>{categoryGroups[4]}</b>", f"<b>{categoryGroups[5]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      # [surveyData[surveyData['sex'] == 'männlich'], surveyData[surveyData['sex'] == 'nicht angegeben'], surveyData[surveyData['sex'] == 'weiblich']]],
        ['<b>Gesamt</b>','<b>Liste favorisiert</b>', '<b>Chord favorisiert</b>', '<b>Map favorisiert</b>'],
        [dfAll.iloc[0]['count'], dfListe.iloc[0]['count'], dfChord.iloc[0]['count'], 0],
        [dfAll.iloc[1]['count'], dfListe.iloc[1]['count'], 0, 0],
        [dfAll.iloc[2]['count'], dfListe.iloc[2]['count'], dfChord.iloc[1]['count'], dfMap.iloc[0]['count']],
        [dfAll.iloc[3]['count'], dfListe.iloc[3]['count'], dfChord.iloc[2]['count'], dfMap.iloc[1]['count']],
        [dfAll.iloc[4]['count'], dfListe.iloc[4]['count'], dfChord.iloc[3]['count'], dfMap.iloc[2]['count']],
        [dfAll.iloc[5]['count'], 0, dfChord.iloc[4]['count'], dfMap.iloc[3]['count']]
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()


## Internet Level

In [41]:
df = surveyData[['internet-level']]

df = df.groupby(['internet-level'], dropna=False)['internet-level'].count().reset_index(name='count')

# Label strings
chartLabel = f"Internetlevel der Teilnehmer, n={len(surveyData[['internet-level']])}"
categoryGroups = ["Anfänger", "Erfahrene Nutzerin, Erfahrener Nutzer", "Internet-Profi"]

fig = go.Figure(data=[go.Pie( labels=categoryGroups, values=df['count'], sort=False)])
fig.update_traces(textinfo='percent+label', marker=dict(colors=urCdColors))
fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    width=600,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)
fig.show()

# Table mood development
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Internetlevel</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      # [surveyData[surveyData['sex'] == 'männlich'], surveyData[surveyData['sex'] == 'nicht angegeben'], surveyData[surveyData['sex'] == 'weiblich']]],
        ['<b>Anzahl</b>'],
        [df.iloc[0]['count']],
        [df.iloc[1]['count']],
        [df.iloc[2]['count']]
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

### Internet Level To Choice

In [12]:
dfAll = surveyData[['internet-level', 'highest-preferenced-viz']]
dfAll = dfAll.groupby(['internet-level'], dropna=False)['internet-level'].count().reset_index(name='count')

dfListe = surveyData.loc[surveyData['highest-preferenced-viz'] == 'Liste'][['internet-level', 'highest-preferenced-viz']]
dfListe = dfListe.groupby(['internet-level'], dropna=False)['internet-level'].count().reset_index(name='count')

dfChord = surveyData.loc[surveyData['highest-preferenced-viz'] == 'Chord'][['internet-level', 'highest-preferenced-viz']]
dfChord = dfChord.groupby(['internet-level'], dropna=False)['internet-level'].count().reset_index(name='count')

dfMap = surveyData.loc[surveyData['highest-preferenced-viz'] == 'Map'][['internet-level', 'highest-preferenced-viz']]
dfMap = dfMap.groupby(['internet-level'], dropna=False)['internet-level'].count().reset_index(name='count')

# Label strings
chartLabel = f"Internetlevel der Teilnehmer:innen, n={len(surveyData[['internet-level']])}"
categoryGroups = ["Anfänger", "Erfahrene Nutzerin, Erfahrener Nutzer", "Internet-Profi"]

fig = make_subplots(rows=2, cols=3, 
    specs=[
        [None, {'type':'domain'}, None],
        [{'type':'domain'}, {'type':'domain'}, {'type':'domain'}],
    ],
    subplot_titles=("Gesamt","Liste favorisiert", "Chord  favorisiert",  "Map favorisiert"))

fig.add_trace(go.Pie(labels=categoryGroups, values=dfAll['count'], sort=False, direction='clockwise', marker_colors=urCdColors),
              1, 2)
fig.add_trace(go.Pie(labels=categoryGroups, values=dfListe['count'], sort=False, direction='clockwise', marker_colors=urCdColors),
              2, 1)
fig.add_trace(go.Pie(labels=categoryGroups, values=dfChord['count'], sort=False, direction='clockwise', marker_colors=urCdColors),
              2, 2)
fig.add_trace(go.Pie(labels=categoryGroups, values=dfMap['count'], sort=False, direction='clockwise', marker_colors=urCdColors),
              2, 3)

fig.update_traces(textinfo='percent+label')
fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    height=850,
    #legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)
fig.update(layout_showlegend=True)
fig.update_annotations(yshift=70)
fig.show()


# Table mood development
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Internetlevel</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      # [surveyData[surveyData['sex'] == 'männlich'], surveyData[surveyData['sex'] == 'nicht angegeben'], surveyData[surveyData['sex'] == 'weiblich']]],
        ['<b>Gesamt</b>','<b>Liste</b>', '<b>Chord</b>', '<b>Map</b>'],
        [dfAll.iloc[0]['count'], dfListe.iloc[0]['count'], dfChord.iloc[0]['count'], dfMap.iloc[0]['count']],
        [dfAll.iloc[1]['count'], dfListe.iloc[1]['count'], dfChord.iloc[1]['count'], dfMap.iloc[1]['count']],
        [dfAll.iloc[2]['count'], dfListe.iloc[2]['count'], dfChord.iloc[2]['count'], dfMap.iloc[2]['count']]
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()


## Internet Usage Setting

In [13]:
df = surveyData[['browsing-type']]

df = df.groupby(['browsing-type'], dropna=False)['browsing-type'].count().reset_index(name='count')

# Label strings
chartLabel = f"Internetnutzung nach Setting, n={len(surveyData[['browsing-type']])}"
categoryGroups = ["Arbeit", "Beides etwa gleich", "Privat"]

fig = go.Figure(data=[go.Pie( labels=categoryGroups, values=df['count'], sort=False)])
fig.update_traces(textinfo='percent+label', marker=dict(colors=urCdColors))
fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    width=600,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)
fig.show()

# Table mood development
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Internetnutzungssetting</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      # [surveyData[surveyData['sex'] == 'männlich'], surveyData[surveyData['sex'] == 'nicht angegeben'], surveyData[surveyData['sex'] == 'weiblich']]],
        ['<b>Anzahl</b>'],
        [df.iloc[0]['count']],
        [df.iloc[1]['count']],
        [df.iloc[2]['count']]
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

### Internet Usage Setting To Choice

In [14]:
dfAll = surveyData[['browsing-type', 'highest-preferenced-viz']]
dfAll = dfAll.groupby(['browsing-type'], dropna=False)['browsing-type'].count().reset_index(name='count')

dfListe = surveyData.loc[surveyData['highest-preferenced-viz'] == 'Liste'][['browsing-type', 'highest-preferenced-viz']]
dfListe = dfListe.groupby(['browsing-type'], dropna=False)['browsing-type'].count().reset_index(name='count')

dfChord = surveyData.loc[surveyData['highest-preferenced-viz'] == 'Chord'][['browsing-type', 'highest-preferenced-viz']]
dfChord = dfChord.groupby(['browsing-type'], dropna=False)['browsing-type'].count().reset_index(name='count')

dfMap = surveyData.loc[surveyData['highest-preferenced-viz'] == 'Map'][['browsing-type', 'highest-preferenced-viz']]
dfMap = dfMap.groupby(['browsing-type'], dropna=False)['browsing-type'].count().reset_index(name='count')

# Label strings
chartLabel = f"Internetnutzung nach Setting, n={len(surveyData[['browsing-type']])}"
categoryGroups = ["Arbeit", "Beides etwa gleich", "Privat"]

fig = make_subplots(rows=2, cols=3, 
    specs=[
        [None, {'type':'domain'}, None],
        [{'type':'domain'}, {'type':'domain'}, {'type':'domain'}],
    ],
    subplot_titles=("Gesamt","Liste favorisiert", "Chord  favorisiert",  "Map favorisiert"))

fig.add_trace(go.Pie(labels=categoryGroups, values=dfAll['count'], sort=False, direction='clockwise', marker_colors=urCdColors),
              1, 2)
fig.add_trace(go.Pie(labels=categoryGroups, values=dfListe['count'], sort=False, direction='clockwise', marker_colors=urCdColors),
              2, 1)
fig.add_trace(go.Pie(labels=[categoryGroups[1], categoryGroups[2]], values=dfChord['count'], sort=False, direction='clockwise', marker_colors=[urCdColors[1], urCdColors[2]]),
              2, 2)
fig.add_trace(go.Pie(labels=[categoryGroups[1], categoryGroups[2]], values=dfMap['count'], sort=False, direction='clockwise', marker_colors=[urCdColors[1], urCdColors[2]]),
              2, 3)

fig.update_traces(textinfo='percent+label')
fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    height=850,
    #legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)
fig.update(layout_showlegend=True)
fig.update_annotations(yshift=70)
fig.show()


# Table mood development
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Internetnutzungssetting</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      # [surveyData[surveyData['sex'] == 'männlich'], surveyData[surveyData['sex'] == 'nicht angegeben'], surveyData[surveyData['sex'] == 'weiblich']]],
        ['<b>Gesamt</b>','<b>Liste</b>', '<b>Chord</b>', '<b>Map</b>'],
        [dfAll.iloc[0]['count'], dfListe.iloc[0]['count'], 0, 0],
        [dfAll.iloc[1]['count'], dfListe.iloc[1]['count'], dfChord.iloc[0]['count'], dfMap.iloc[0]['count']],
        [dfAll.iloc[2]['count'], dfListe.iloc[2]['count'], dfChord.iloc[1]['count'], dfMap.iloc[1]['count']]
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

## Internet Usage

### Stacked Usage

In [15]:
usageSelection = [
        "internet-usage-e-mails",
        "internet-usage-messenger",
        "internet-usage-encyclopedia",
        "internet-usage-youtube",
        "internet-usage-weather",
        "internet-usage-products-and-prices",
        "internet-usage-banking",
        "internet-usage-maps",
        "internet-usage-voip",
        "internet-usage-social-media-networks",
        "internet-usage-music-streaming",
        "internet-usage-video-streaming",
        "internet-usage-news",
        "internet-usage-podcasts",
        "internet-usage-blogs",
        "internet-usage-contact-portals",
        "internet-usage-news-archives",
        "internet-usage-job-portals",
        "internet-usage-twitter"
    ]

namesMap = {
    "internet-usage-e-mails": "E-Mails",
    "internet-usage-messenger": "Messenger",
    "internet-usage-encyclopedia": "Enzyklopädien",
    "internet-usage-youtube": "Youtube",
    "internet-usage-weather": "Wetterdienste",
    "internet-usage-products-and-prices": "Produkt- und Preisvergleiche",
    "internet-usage-banking": "Onlinebanking",
    "internet-usage-maps": "Kartenanwendungen",
    "internet-usage-voip": "Internettelefonie",
    "internet-usage-social-media-networks": "Social Media",
    "internet-usage-music-streaming": "Musikstreaming",
    "internet-usage-video-streaming": "Videostreaming",
    "internet-usage-news": "Nachrichten",
    "internet-usage-podcasts": "Podcasts",
    "internet-usage-blogs": "Blogs",
    "internet-usage-contact-portals": "Kontaktbörsen",
    "internet-usage-news-archives": "Zeitungsarchive",
    "internet-usage-job-portals": "Jobportale",
    "internet-usage-twitter": "Twitter"
}

# Label strings
chartLabel = f"Internetnutzung nach Art, n={len(surveyData[['internet-usage-e-mails']])}"

df = surveyData[usageSelection]

# convertedDf = pd.DataFrame(columns=['usage-type','frequency'])
convertedDf = pd.DataFrame(columns=['usage-type', 'frequency'])

# Iterate over all columns
for column in df:
    # Iterate over all rows
    rowTypeCountsDf = pd.DataFrame(columns=['usage-type', 'frequency'])

    for row in df[column]:
        # Build new data to add
        newRow = pd.DataFrame([[column, row]], columns=['usage-type', 'frequency'])
        # Add new Data
        rowTypeCountsDf = pd.concat([rowTypeCountsDf, newRow], ignore_index=True)
    # Build counts per column    
    rowTypeCountsDf = rowTypeCountsDf.groupby(['usage-type', 'frequency'], dropna=False)['frequency'].count().reset_index(name='count')
    # Add data to sum df
    convertedDf = pd.concat([convertedDf, rowTypeCountsDf], ignore_index=True)

convertedDf = convertedDf.replace({"usage-type": namesMap})

fig = px.bar(convertedDf, y="usage-type", x="count", color="frequency", title="Internetnutzung nach Art", 
    color_discrete_sequence =urCdColors*3, 
    orientation='h', 
    category_orders={
        'frequency': ['Nie', 'Seltener', 'Mehrmals pro Monat', 'Mehrmals pro Woche', 'Täglich'],
        'usage-type': [
            'Jobportale', 
            'Kontaktbörsen', 
            'Zeitungsarchive', 
            'Blogs', 
            'Kartenanwendungen', 
            'Twitter', 
            'Produkt- und Preisvergleiche',
            'Podcasts', 
            'Onlinebanking', 
            'Internettelefonie', 
            'Nachrichten', 
            'Enzyklopädien', 
            'Wetterdienste', 
            'Videostreaming', 
            'Social Media', 
            'E-Mails', 
            'Musikstreaming', 
            'Youtube', 
            'Messenger'
        ]
    }, 
    text_auto=True
)
fig.update_layout(
    title=chartLabel,
    yaxis_title='Art',
    xaxis_title='Teilnehmerantwort',
    height=850,
    legend_title="Häufigkeit",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()


## Absolute Usage

In [16]:
# Label strings
chartLabel = f"Internetnutzung nach Art in Prozenten, n={len(surveyData[['browsing-type']])}"

# Transform frequency in values
convertedDf = convertedDf.replace(['Nie', 'Seltener', 'Mehrmals pro Monat', 'Mehrmals pro Woche', 'Täglich'], [0, 0, 0, 1, 1])

# Calculates absolute frequency
convertedDf['absolute-frequency'] = convertedDf.apply(lambda row: row['frequency'] * row['count'], axis=1)
convertedDf
# Sum absolute frequency values
sumDf = convertedDf.groupby('usage-type')['absolute-frequency'].sum().reset_index()
# Sort values descending 
sumDf = sumDf.sort_values(by='absolute-frequency', ignore_index=True)
# Create percentage Value ‹‹‹ probably not necessary cause Plotly does it for me
totalFrequencySum = len(surveyData[['browsing-type']]) 
sumDf['frequency-percentage'] = sumDf.apply(lambda row: row['absolute-frequency'] / totalFrequencySum * 100, axis=1)
sumDf['label'] = sumDf.apply(lambda row: "" + str(round(row['frequency-percentage'], 1)) + " % (" + str(row['absolute-frequency']) + ")", axis=1)
sumDf = sumDf.replace({"usage-type": namesMap})

fig = px.bar(sumDf, y="usage-type", x="absolute-frequency", title="Internetangebote, die am häufigsten unter den Teilnehmer:innen genutzt werden (mind. 1 x pro Woche)", 
    color_discrete_sequence =urCdColors*3, 
    orientation='h', 
    category_orders={
        'usage-type': sumDf['usage-type'].tolist()
    }, 
    text='label'
)

fig.update_traces(textposition="outside", cliponaxis=False)
fig.update_layout(
    title=chartLabel,
    yaxis_title='Art',
    xaxis_title='Teilnehmerantwort',
    height=850,
    legend_title="Häufigkeit",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()


## Nutrition Data
### Meal Preparing A Week

In [17]:
df = surveyData[['meal-preparation-a-week']]
df = df.groupby(['meal-preparation-a-week'], dropna=False)['meal-preparation-a-week'].count().reset_index(name='count')
# Label strings
chartLabel = f"Häufigkeit der pro Woche zubereiteten Mahlzeiten, n={len(surveyData[['meal-preparation-a-week']])}"
categoryGroups = ["< 1 Mal", "1–2 Mal", "3–4 Mal", "5–6 Mal", "Täglich"]
categoryGroupsOrder = CategoricalDtype(
    categoryGroups, 
    ordered=True
)

df['meal-preparation-a-week'] = df['meal-preparation-a-week'].astype(categoryGroupsOrder)
df = df.sort_values('meal-preparation-a-week', ignore_index=True)

fig = go.Figure(data=[go.Pie( labels=categoryGroups, values=df['count'], sort=False, direction='clockwise')])
fig.update_traces(textinfo='percent+label', marker=dict(colors=urCdColors))
fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    width=850,
    # height=650,
    legend_title="Anzahl pro Woche zubereiteter Mahlzeiten",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)
fig.show()

# Table sex
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Häufigkeit zubereiteter Mahlzeiten</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>", f"<b>{categoryGroups[3]}</b>", f"<b>{categoryGroups[4]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      # [surveyData[surveyData['sex'] == 'männlich'], surveyData[surveyData['sex'] == 'nicht angegeben'], surveyData[surveyData['sex'] == 'weiblich']]],
        ['<b>Anzahl</b>'],
        [len(surveyData[surveyData['meal-preparation-a-week'] == categoryGroups[0]])],
        [len(surveyData[surveyData['meal-preparation-a-week'] == categoryGroups[1]])],
        [len(surveyData[surveyData['meal-preparation-a-week'] == categoryGroups[2]])],
        [len(surveyData[surveyData['meal-preparation-a-week'] == categoryGroups[3]])],
        [len(surveyData[surveyData['meal-preparation-a-week'] == categoryGroups[4]])]
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

### Interest in Nutrition

In [18]:
df = surveyData[['interest-in-nutrition']]
df = df.groupby(['interest-in-nutrition'], dropna=False)['interest-in-nutrition'].count().reset_index(name='count')

# Label strings
chartLabel = f"Interesse am Thema Ernährung, n={len(surveyData[['interest-in-nutrition']])}"
categoryGroups = ["Gar nicht", "Wenig", "Mittelmäßig", "Ziemlich", "Sehr"]
categoryGroupsOrder = CategoricalDtype(
    categoryGroups, 
    ordered=True
)

df['interest-in-nutrition'] = df['interest-in-nutrition'].astype(categoryGroupsOrder)
df = df.sort_values('interest-in-nutrition', ignore_index=True)

fig = go.Figure(data=[go.Pie( labels=categoryGroups, values=df['count'], sort=False, direction='clockwise')])
fig.update_traces(textinfo='percent+label', marker=dict(colors=urCdColors))
fig.update_layout(
    title=chartLabel,
    width=850,
    # height=600,
    # yaxis_title=yLabel,
    legend_title="Staffelung des Interesses",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)
fig.show()

# Table
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Interesse an Ernährung</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>", f"<b>{categoryGroups[3]}</b>", f"<b>{categoryGroups[4]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      # [surveyData[surveyData['sex'] == 'männlich'], surveyData[surveyData['sex'] == 'nicht angegeben'], surveyData[surveyData['sex'] == 'weiblich']]],
        ['<b>Anzahl</b>'],
        [len(surveyData[surveyData['interest-in-nutrition'] == categoryGroups[0]])],
        [len(surveyData[surveyData['interest-in-nutrition'] == categoryGroups[1]])],
        [len(surveyData[surveyData['interest-in-nutrition'] == categoryGroups[2]])],
        [len(surveyData[surveyData['interest-in-nutrition'] == categoryGroups[3]])],
        [len(surveyData[surveyData['interest-in-nutrition'] == categoryGroups[4]])]
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

### Nutrition Habit

In [42]:
df = surveyData[['nutrition-habit', 'nutrition-habit-other']]

# Replacements
df.at[14,'nutrition-habit']="Kein Fisch"
df.at[58,'nutrition-habit']="Flexitarisch"
df.at[64,'nutrition-habit']="Flexitarisch"
df.at[72,'nutrition-habit']="Flexitarisch"
df.at[79,'nutrition-habit']="Fleisch"
df.at[80,'nutrition-habit']="Regional, direkt vom Erzeuger, Selbstversorger"
df.at[101,'nutrition-habit']="Flexitarisch"
df.at[128,'nutrition-habit']="Flexitarisch"
df.at[152,'nutrition-habit']="Flexitarisch"
df.at[187,'nutrition-habit']="Diabetikererecht"
df.at[198,'nutrition-habit']="Mediterran"

df = df.groupby(['nutrition-habit'], dropna=False)['nutrition-habit'].count().reset_index(name='count')

# # Label strings
chartLabel = f"Ernährungsform der Teilnehmer, n={len(surveyData[['nutrition-habit']])}"

df = df.sort_values('count', ascending=True, ignore_index=True)

fig = go.Figure(data=[go.Pie( labels=df['nutrition-habit'], values=df['count'], sort=False, direction='clockwise')])
fig.update_traces(textinfo='percent+label', marker=dict(colors=urCdColors))
fig.update_layout(
    title={
        'text': chartLabel,
        'y':0.95,
        # 'x':0,
        'xanchor': 'left',
        'yanchor': 'top'
    },
    # yaxis_title=yLabel,
    width=850,
    height=850,
    legend_title="Ernährungsform",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=15,
        # color="RebeccaPurple"
    )
)
fig.show()
df[['nutrition-habit', 'count']]
# Table
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Ernährungsform</b>', f"<b>Anzahl</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[df['nutrition-habit'], df['count']],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

### Nutrition Diet

In [20]:
df = surveyData[['diet', 'diet-other']]

# # Replacements
df.at[19,'diet']="Periodenfasten"
df.at[21,'diet']="Intervallfasten"
df.at[105,'diet']="Intervallfasten"
df.at[123,'diet']="Intervallfasten"
df.at[182,'diet']="Intervallfasten"
df.at[118,'diet']="Kalorienzählen"
df.at[124,'diet']="Kalorienzählen"
df.at[132,'diet']="Kalorienzählen"
df.at[200,'diet']="Low Carb"

df = df.groupby(['diet'], dropna=False)['diet'].count().reset_index(name='count')

# Label strings
chartLabel = f"Diäten der Teilnehmer, n={len(surveyData[['diet']])}"

df = df.sort_values('count', ascending=True, ignore_index=True)

categoryGroups = df['diet']
categoryGroups.at[4]="Keine"

fig = go.Figure(data=[go.Pie( labels=categoryGroups, values=df['count'], sort=False, direction='clockwise')])
fig.update_traces(textinfo='percent+label', marker=dict(colors=urCdColors))
fig.update_layout(
    title=chartLabel,
    # yaxis_title=yLabel,
    width=850,
    # height=650,
    legend_title="Diät",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)
fig.show()

# Table
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Diät</b>', f"<b>Anzahl</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[df['diet'], df['count']],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

### Nutrition Intolerance

In [21]:
df = surveyData[['nutrition-intolerance', 'nutrition-intolerance-other']]

# Replacements
df.at[1,'nutrition-intolerance']="Meeresfrüchte"
df.at[14,'nutrition-intolerance']="Apfel"
df.at[48,'nutrition-intolerance']="Reizdarm"
df.at[80,'nutrition-intolerance']="Hohe Fetthaltigkeit"
df.at[93,'nutrition-intolerance']="Apfel"
df.at[87,'nutrition-intolerance']="Histamin"
df.at[146,'nutrition-intolerance']="Amoxicillin"
df.at[176,'nutrition-intolerance']="Einzelne Früchte"

df = df.groupby(['nutrition-intolerance'], dropna=False)['nutrition-intolerance'].count().reset_index(name='count')

# Label strings
chartLabel = f"Unverträglichkeiten der Teilnehmer, n={len(surveyData[['nutrition-intolerance']])}"

df = df.sort_values('count', ascending=True, ignore_index=True)

fig = go.Figure(data=[go.Pie( labels=df['nutrition-intolerance'], values=df['count'], sort=False, direction='clockwise')])
fig.update_traces(textinfo='percent+label', marker=dict(colors=urCdColors))
fig.update_layout(
    title=chartLabel,
    # yaxis_title=yLabel,
    width=1080,
    height=850,
    legend_title="Diät",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=15,
        # color="RebeccaPurple"
    )
)
fig.show()

# Table
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Unverträglichkeit</b>', f"<b>Anzahl</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[df['nutrition-intolerance'], df['count']],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

### Nutrition Easiness

In [22]:
df = surveyData[['nutrition-easiness']]

df = df.groupby(['nutrition-easiness'], dropna=False)['nutrition-easiness'].count().reset_index(name='count')

# Label strings
chartLabel = f"Einfachheit zur Umsetzung eigener Ernährungswünsche der Teilnehmer, n={len(surveyData[['nutrition-easiness']])}"
categoryGroups = ["Sehr schwer", "Schwer", "Es geht", "Leicht", "Sehr leicht"]
categoryGroupsOrder = CategoricalDtype(
    categoryGroups, 
    ordered=True
)

df['nutrition-easiness'] = df['nutrition-easiness'].astype(categoryGroupsOrder)
df = df.sort_values('nutrition-easiness', ignore_index=True)

fig = go.Figure(data=[go.Pie( labels=categoryGroups, values=df['count'], sort=False, direction='clockwise')])
fig.update_traces(textinfo='percent+label', marker=dict(colors=urCdColors))
fig.update_layout(
    title=chartLabel,
    # yaxis_title=yLabel,
    width=850,
    # height=850,
    legend_title="Einfachheit",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)
fig.show()

# Table
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Einfachheit</b>', f"<b>Anzahl</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[df['nutrition-easiness'], df['count']],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

## Tasks

### Video Time

In [23]:
# Standard deviation time and mood
# df = surveyData.loc[surveyData['duration-group-tasks-intro'] < 4000][['duration-group-tasks-intro', 'highest-preferenced-viz']]
df = surveyData.loc[surveyData['duration-group-tasks-intro'] < 1200][['duration-group-tasks-intro', 'highest-preferenced-viz']]

dfSorted = surveyData[['duration-group-tasks-intro', 'highest-preferenced-viz']].sort_values(by=['duration-group-tasks-intro'])
# print(dfSorted)
# dfTooFast = surveyData.loc[surveyData['duration-group-tasks-intro'] < 250][['duration-group-tasks-intro', 'highest-preferenced-viz']]
# print(dfTooFast)

# Label strings
chartLabel = f"Dauer für Videoteil (Video Standard 8:19 Minuten = 499 Sekunden), n={len(df)} von {len(surveyData)} (8 Ausreißer > 20 Minuten ausgeschlossen)"
categoryGroups = ["alle Teilnehmer:innen", "Liste favorisiert", "Chord  favorisiert", "Map  favorisiert"]
yLabel = "Dauer der Videoaufgabe in Sekunden"
fig = go.Figure()

dfAll = df['duration-group-tasks-intro']
dfList = df[df['highest-preferenced-viz'] == 'Liste']['duration-group-tasks-intro']
dfChord = df[df['highest-preferenced-viz'] == 'Chord']['duration-group-tasks-intro']
dfMap = df[df['highest-preferenced-viz'] == 'Map']['duration-group-tasks-intro']
# Video task time all
fig.add_trace(go.Box(y=dfAll, name=categoryGroups[0],
                marker_color = urCdColors[0]))
# Age preferred List
fig.add_trace(go.Box(y=dfList, name=categoryGroups[1],
                marker_color = urCdColors[1]))
# Age preferred Chord
fig.add_trace(go.Box(y=dfChord, name=categoryGroups[2],
                marker_color = urCdColors[2]))
# Age preferred Map
fig.add_trace(go.Box(y=dfMap, name=categoryGroups[3],
                marker_color = urCdColors[3]))
fig.update_layout(
    title=chartLabel,
    xaxis_title="Zeitverteilung der Videoaufgabe nach Präferenz",
    yaxis_title=yLabel,
    # width=1080,
    height=850,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=18,
        # color="RebeccaPurple"
    )
)

fig.update_layout(title_text=chartLabel)
fig.show()


# Table video task time
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Kennwert</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>", f"<b>{categoryGroups[3]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['<b>Maximum</b>', '<b>Oberes Quartil</b>', '<b>Median</b>', '<b>Unteres Quartil</b>', '<b>Minimum</b>'],
      # TODO: GET DIRECT VALUES
      [round(dfAll.describe()['max']), round(dfAll.describe()['75%']), round(dfAll.describe()['50%']), round(dfAll.describe()['25%']), round(dfAll.describe()['min'])],
      [round(dfList.describe()['max']), round(dfList.describe()['75%']), round(dfList.describe()['50%']), round(dfList.describe()['25%']), round(dfList.describe()['min'])],
      [round(dfChord.describe()['max']), round(dfChord.describe()['75%']), round(dfChord.describe()['50%']), round(dfChord.describe()['25%']), round(dfChord.describe()['min'])],
      [round(dfMap.describe()['max']), round(dfMap.describe()['75%']), round(dfMap.describe()['50%']), round(dfMap.describe()['25%']), round(dfMap.describe()['min'])]
    ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor]*6],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel + " – gerundete Sekunden",
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()


### Task Times

In [24]:
# Clean Data because of no Inteaction in Tasks and too short duration-task times or duration-over-all
# Times for  All, List, Chord, Map as Mean 462.062222	27.698889	17.613333	26.521111
# Also no qualitative Feedback at all
dataToClean = surveyData;

filterArray = [
  109,
  113,
  124,
  130,
  138,
  145,
  157,
  158,
  191,
]
timeCleanedData = dataToClean.drop(filterArray)

dfList = timeCleanedData['duration-group-list-task']
dfChord = timeCleanedData['duration-group-chord-task']
dfMap = timeCleanedData['duration-group-map-task']

# For Comparison a second Test was conducted with cleaned outliiers who took more than 1000 seconds for each task
# dfNoOutliers  = timeCleanedData.loc[
#   (timeCleanedData['duration-group-list-task'] < 1000) & (timeCleanedData['duration-group-chord-task'] < 1000) & (timeCleanedData['duration-group-map-task'] < 1000)
# ][['id', 'duration-over-all', 'duration-group-list-task', 'duration-group-chord-task', 'duration-group-map-task']]

# dfList = dfNoOutliers['duration-group-list-task']
# dfChord = dfNoOutliers['duration-group-chord-task']
# dfMap = dfNoOutliers['duration-group-map-task']


# Label strings
chartLabel = f"Dauer der Aufgaben, n={len(timeCleanedData)}"
categoryGroups = ["Aufgabe Liste", " Aufgabe Chord", "Aufgabe Map"]
yLabel = "Dauer der Aufgaben in Sekunden"
fig = go.Figure()

# List task time 
fig.add_trace(go.Box(y=dfList, name=categoryGroups[0],
                marker_color = urCdColors[0]))
# Chord task time 
fig.add_trace(go.Box(y=dfChord, name=categoryGroups[1],
                marker_color = urCdColors[1]))
# Map task time 
fig.add_trace(go.Box(y=dfMap, name=categoryGroups[2],
                marker_color = urCdColors[2]))

# fig.update_traces(quartilemethod="exclusive")
fig.update_layout(
    title=chartLabel,
    xaxis_title="Zeitverteilung der Aufgaben nach Art",
    yaxis_title=yLabel,
    height=850,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=18,
        # color="RebeccaPurple"
    )
)

print("## LISTE")
print(dfList.describe())
print("## Chord")
print(dfChord.describe())
print("## Map")
print(dfMap.describe())
print("## List-Chord")
print(ttest_rel(dfList, dfChord))
print("## List-Map")
print(ttest_rel(dfList, dfMap))
print("## Chord-Map")
print(ttest_rel(dfChord, dfMap))

fig.update_layout(title_text=chartLabel)
fig.show()


# Table video task time
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Kennwert</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['<b>Maximum</b>', '<b>Oberes Quartil</b>', '<b>Median</b>', '<b>Unteres Quartil</b>', '<b>Minimum</b>'],
      [round(dfList.describe()['max']), round(dfList.describe()['75%']), round(dfList.describe()['50%']), round(dfList.describe()['25%']), round(dfList.describe()['min'])],
      [round(dfChord.describe()['max']), round(dfChord.describe()['75%']), round(dfChord.describe()['50%']), round(dfChord.describe()['25%']), round(dfChord.describe()['min'])],
      [round(dfMap.describe()['max']), round(dfMap.describe()['75%']), round(dfMap.describe()['50%']), round(dfMap.describe()['25%']), round(dfMap.describe()['min'])]
    ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor]*6],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel + " – gerundete Sekunden",
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()


## LISTE
count     194.000000
mean      220.105773
std       197.427398
min        12.560000
25%       141.502500
50%       202.035000
75%       262.150000
max      2535.470000
Name: duration-group-list-task, dtype: float64
## Chord
count     194.000000
mean      210.327938
std       165.515879
min         8.940000
25%       132.167500
50%       186.105000
75%       254.712500
max      1613.570000
Name: duration-group-chord-task, dtype: float64
## Map
count     194.000000
mean      242.823814
std       153.816211
min        14.510000
25%       155.157500
50%       222.415000
75%       296.592500
max      1278.530000
Name: duration-group-map-task, dtype: float64
## List-Chord
TtestResult(statistic=0.5900092983050621, pvalue=0.5558745514411978, df=193)
## List-Map
TtestResult(statistic=-1.4292438955689615, pvalue=0.15455072772754677, df=193)
## Chord-Map
TtestResult(statistic=-2.6046311653294434, pvalue=0.00991340539686242, df=193)


### Task Completion Rate

Sucess Rate after Nielsen

https://www.nngroup.com/articles/success-rate-the-simplest-usability-metric/

Nielsen, Jakob/Raluca Budiu: »Success Rate: The Simplest Usability Metric«, https://www.nngroup.com/articles/success-rate-the-simplest-usability-metric/, 2001, aufgerufen am: 22.10.21.

* complete success: the user places the order with no error, exactly as specified
* success with one minor issue: the user places the order but omits the gift message or orders the wrong flowers
* success with a major issue: the user places the order but enters the wrong date or delivery address  
* failure: the user is not able to place the order

### Liste 
* Die Zwiebel: Misserfolg
* Oishii: Erfolg mit größeren Problemen 
* Vapiano: Erfolg mit kleineren Problemen
* American Diner Durlach: Vollständiger Erfolg

### Chord
* Allvitalis Bar: Misserfolg
* Zur Alten Hackerei: Erfolg mit größeren Problemen
* Oval: Erfolg mit größeren Problemen
* DOM - Grill Kitchen Bar: Erfolg mit kleineren Problemen
* Scruffy's Irish Pub: Vollständiger Erfolg

### Map
* Walksches Haus: Misserfolg
* Exil Cölner: Erfolg mit größeren Problemen,
* s' Häusle: Erfolg mit größeren Problemen,
* Restaurant Beim Schupi: Erfolg mit kleineren Problemen,
* Kesselhaus: Erfolg mit kleineren Problemen,
* Restaurant Kaiserhof: Vollständiger Erfolg


In [25]:
listNamesMap = {
    "Die Zwiebel": "Misserfolg",
    "Oishii": "Erfolg mit größeren Problemen",
    "Vapiano": "Erfolg mit kleineren Problemen",
    "American Diner Durlach": "Vollständiger Erfolg"
}
chordNamesMap = {
    "Allvitalis Bar": "Misserfolg",
    "Zur Alten Hackerei": "Erfolg mit größeren Problemen",
    "Oval": "Erfolg mit größeren Problemen",
    "DOM - Grill Kitchen Bar": "Erfolg mit kleineren Problemen",
    "Scruffy's Irish Pub": "Vollständiger Erfolg"
}
mapNamesMap = {
    "Walksches Haus": "Misserfolg",
    "Exil Cölner": "Erfolg mit größeren Problemen",
    "s' Häusle": "Erfolg mit größeren Problemen",
    "Restaurant Beim Schupi": "Erfolg mit kleineren Problemen",
    "Kesselhaus": "Erfolg mit kleineren Problemen",
    "Restaurant Kaiserhof": "Vollständiger Erfolg"
}

filterArray = [
  109,
  113,
  124,
  130,
  138,
  145,
  157,
  158,
  191,
]
timeCleanedData = surveyData.drop(filterArray)

# Label strings
chartLabel = f"Erfolgsrate nach Aufgabe, n={len(timeCleanedData[['list-restaurant-choice']])}"

# List
dfList = timeCleanedData[['list-restaurant-choice']]
dfListRenamed = dfList.replace({"list-restaurant-choice": listNamesMap})

dfListRenamed = dfListRenamed.groupby(['list-restaurant-choice'], dropna=False)['list-restaurant-choice'].count().reset_index(name='count')
dfListRenamed.insert(0, 'task', 'Liste')
dfListRenamed.columns = ['task', 'success-level', 'count']

# Chord
dfChord = timeCleanedData[['chord-restaurant-choice']]
dfChordRenamed = dfChord.replace({"chord-restaurant-choice": chordNamesMap})

dfChordRenamed = dfChordRenamed.groupby(['chord-restaurant-choice'], dropna=False)['chord-restaurant-choice'].count().reset_index(name='count')
dfChordRenamed.insert(0, 'task', 'Chord')
dfChordRenamed.columns = ['task', 'success-level', 'count']

# Map
dfMap = timeCleanedData[['map-restaurant-choice']]
dfMapRenamed = dfMap.replace({"map-restaurant-choice": mapNamesMap})

dfMapRenamed = dfMapRenamed.groupby(['map-restaurant-choice'], dropna=False)['map-restaurant-choice'].count().reset_index(name='count')
dfMapRenamed.insert(0, 'task', 'Map')
dfMapRenamed.columns = ['task', 'success-level', 'count']


stackedTaskDf = pd.concat([dfListRenamed, dfChordRenamed, dfMapRenamed], ignore_index=True)
stackedTaskDf['label'] = stackedTaskDf.apply(lambda row: "" + str(round(row['count']/len(timeCleanedData[['list-restaurant-choice']])*100, 1)) + " % (" + str(row['count']) + ")", axis=1)

fig = px.bar(stackedTaskDf, y="task", x="count", color="success-level", title="Erfolgsrate nach Aufgabe", 
    text="label",
    color_discrete_sequence =urCdColors*3, 
    orientation='h', 
    category_orders={
        'success-level': ["Misserfolg", "Erfolg mit größeren Problemen", "Erfolg mit kleineren Problemen", "Vollständiger Erfolg"],
        'task': [
            'Liste', 
            'Chord', 
            'Map', 
        ]
    },
)
fig.update_layout(
    title=chartLabel,
    yaxis_title='Aufgabe',
    xaxis_title='Erfolgsrate',
    # width=1080,
    height=850,
    # margin=dict(
    #     pad=40
    # ),
    legend_title="Erfolsrate in Stufen",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=24,
        # color="RebeccaPurple"
    )
)

fig.show()

# Tabellen
# Liste
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Erfolgrate</b>', f"<b>Anzahl</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
        ["Misserfolg", "Erfolg mit größeren Problemen", "Erfolg mit kleineren Problemen", "Vollständiger Erfolg"],
        [
            dfListRenamed[dfListRenamed['success-level'] == 'Misserfolg']['count'].values[0],
            dfListRenamed[dfListRenamed['success-level'] == 'Erfolg mit größeren Problemen']['count'].values[0],
            dfListRenamed[dfListRenamed['success-level'] == 'Erfolg mit kleineren Problemen']['count'].values[0],
            dfListRenamed[dfListRenamed['success-level'] == 'Vollständiger Erfolg']['count'].values[0],
        ]
    ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title="Erfolgsrate Aufgabe: Liste",
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

# Chord
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Erfolgrate</b>', f"<b>Anzahl</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
        ["Misserfolg", "Erfolg mit größeren Problemen", "Erfolg mit kleineren Problemen", "Vollständiger Erfolg"],
        [
            dfChordRenamed[dfChordRenamed['success-level'] == 'Misserfolg']['count'].values[0],
            dfChordRenamed[dfChordRenamed['success-level'] == 'Erfolg mit größeren Problemen']['count'].values[0],
            dfChordRenamed[dfChordRenamed['success-level'] == 'Erfolg mit kleineren Problemen']['count'].values[0],
            dfChordRenamed[dfChordRenamed['success-level'] == 'Vollständiger Erfolg']['count'].values[0],
        ]
    ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title="Erfolgsrate Aufgabe: Chord",
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

# Chord
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Erfolgrate</b>', f"<b>Anzahl</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
        ["Misserfolg", "Erfolg mit größeren Problemen", "Erfolg mit kleineren Problemen", "Vollständiger Erfolg"],
        [
            dfMapRenamed[dfMapRenamed['success-level'] == 'Misserfolg']['count'].values[0],
            dfMapRenamed[dfMapRenamed['success-level'] == 'Erfolg mit größeren Problemen']['count'].values[0],
            dfMapRenamed[dfMapRenamed['success-level'] == 'Erfolg mit kleineren Problemen']['count'].values[0],
            dfMapRenamed[dfMapRenamed['success-level'] == 'Vollständiger Erfolg']['count'].values[0],
        ]
    ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title="Erfolgsrate Aufgabe: Map",
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

### Decision Easiness

In [26]:
filterArray = [
  109,
  113,
  124,
  130,
  138,
  145,
  157,
  158,
  191,
]
timeCleanedData = surveyData.drop(filterArray)

# Label strings
chartLabel = f"Wie leicht ist Ihnen die Entscheidung mit der Listen-Ansicht gefallen? n={len(timeCleanedData[['list-choice-easiness']])}"

# List
dfList = timeCleanedData[['list-choice-easiness']]

dfList = dfList.groupby(['list-choice-easiness'], dropna=False)['list-choice-easiness'].count().reset_index(name='count')
dfList.insert(0, 'task', 'Liste')
dfList.columns = ['task', 'easiness-level', 'count']

# Chord
dfChord = timeCleanedData[['chord-choice-easiness']]

dfChord = dfChord.groupby(['chord-choice-easiness'], dropna=False)['chord-choice-easiness'].count().reset_index(name='count')
dfChord.insert(0, 'task', 'Chord')
dfChord.columns = ['task', 'easiness-level', 'count']

# Map
dfMap = timeCleanedData[['map-choice-easiness']]

dfMap = dfMap.groupby(['map-choice-easiness'], dropna=False)['map-choice-easiness'].count().reset_index(name='count')
dfMap.insert(0, 'task', 'Map')
dfMap.columns = ['task', 'easiness-level', 'count']

stackedTaskDf = pd.concat([dfList, dfChord, dfMap], ignore_index=True)
stackedTaskDf['label'] = stackedTaskDf.apply(lambda row: "" + str(round(row['count']/len(timeCleanedData[['list-choice-easiness']])*100, 1)) + " % (" + str(row['count']) + ")", axis=1)

fig = px.bar(stackedTaskDf, y="task", x="count", color="easiness-level", title="Empfundene Leichtigkeit bei der Entscheidungsfindung (Wie leicht ist es mir gefallen?)", 
    text="label",
    color_discrete_sequence =urCdColors*3, 
    orientation='h', 
    category_orders={
        'easiness-level': ["Sehr schwer", "Schwer", "Neutral", "Leicht", "Sehr leicht"],
        'task': [
            'Liste', 
            'Chord', 
            'Map', 
        ]
    },
)
fig.update_layout(
    title=chartLabel,
    yaxis_title='Aufgabe',
    xaxis_title='Empfundene Leichtigkeit',
    height=850,
    # margin=dict(
    #     pad=40
    # ),
    legend_title="Wie leicht fiel es mir",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=24,
        # color="RebeccaPurple"
    )
)

fig.show()


# Tabellen
# Liste
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Wie leicht viel es mir?</b>', f"<b>Anzahl</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
        ["Sehr schwer", "Schwer", "Neutral", "Leicht", "Sehr leicht"],
        [
            dfList[dfList['easiness-level'] == 'Sehr schwer']['count'].values[0],
            dfList[dfList['easiness-level'] == 'Schwer']['count'].values[0],
            dfList[dfList['easiness-level'] == 'Neutral']['count'].values[0],
            dfList[dfList['easiness-level'] == 'Leicht']['count'].values[0],
            dfList[dfList['easiness-level'] == 'Sehr leicht']['count'].values[0],
        ]
    ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title="Leichtigkeit eine Entscheidung zu Treffen: Liste",
    yaxis_title="Aufgabe",
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

# Chord
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Wie leicht viel es mir?</b>', f"<b>Anzahl</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
        ["Sehr schwer", "Schwer", "Neutral", "Leicht", "Sehr leicht"],
        [
            dfChord[dfChord['easiness-level'] == 'Sehr schwer']['count'].values[0],
            dfChord[dfChord['easiness-level'] == 'Schwer']['count'].values[0],
            dfChord[dfChord['easiness-level'] == 'Neutral']['count'].values[0],
            dfChord[dfChord['easiness-level'] == 'Leicht']['count'].values[0],
            dfChord[dfChord['easiness-level'] == 'Sehr leicht']['count'].values[0],
        ]
    ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title="Leichtigkeit eine Entscheidung zu Treffen: Chord",
    yaxis_title="Aufgabe",
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

# Chord
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Wie leicht viel es mir?</b>', f"<b>Anzahl</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
        ["Sehr schwer", "Schwer", "Neutral", "Leicht", "Sehr leicht"],
        [
            dfMap[dfMap['easiness-level'] == 'Sehr schwer']['count'].values[0],
            dfMap[dfMap['easiness-level'] == 'Schwer']['count'].values[0],
            dfMap[dfMap['easiness-level'] == 'Neutral']['count'].values[0],
            dfMap[dfMap['easiness-level'] == 'Leicht']['count'].values[0],
            dfMap[dfMap['easiness-level'] == 'Sehr leicht']['count'].values[0],
        ]
    ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title="Leichtigkeit eine Entscheidung zu Treffen: : Map",
    yaxis_title="Aufgabe",
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

### Decision Satisfaction

In [27]:
filterArray = [
  109,
  113,
  124,
  130,
  138,
  145,
  157,
  158,
  191,
]
timeCleanedData = surveyData.drop(filterArray)

# Label strings
chartLabel = f"Wie zufrieden sind Sie mit der Entscheidung, in Hinblick auf die beschriebene Situation? n={len(timeCleanedData[['list-choice-satisfaction']])}"

# List
dfList = timeCleanedData[['list-choice-satisfaction']]

dfList = dfList.groupby(['list-choice-satisfaction'], dropna=False)['list-choice-satisfaction'].count().reset_index(name='count')
dfList.insert(0, 'task', 'Liste')
dfList.columns = ['task', 'satisfaction-level', 'count']

# Chord
dfChord = timeCleanedData[['chord-choice-satisfaction']]

dfChord = dfChord.groupby(['chord-choice-satisfaction'], dropna=False)['chord-choice-satisfaction'].count().reset_index(name='count')
dfChord.insert(0, 'task', 'Chord')
dfChord.columns = ['task', 'satisfaction-level', 'count']

# Map
dfMap = timeCleanedData[['map-choice-satisfaction']]

dfMap = dfMap.groupby(['map-choice-satisfaction'], dropna=False)['map-choice-satisfaction'].count().reset_index(name='count')
dfMap.insert(0, 'task', 'Map')
dfMap.columns = ['task', 'satisfaction-level', 'count']

stackedTaskDf = pd.concat([dfList, dfChord, dfMap], ignore_index=True)
stackedTaskDf['label'] = stackedTaskDf.apply(lambda row: "" + str(round(row['count']/len(timeCleanedData[['list-choice-satisfaction']])*100, 1)) + " % (" + str(row['count']) + ")", axis=1)

fig = px.bar(stackedTaskDf, y="task", x="count", color="satisfaction-level", title="Empfundene Zufriedenheit bei der Entscheidungsfindung (Wie leicht ist es mir gefallen?)", 
    text="label",
    color_discrete_sequence =urCdColors*3, 
    orientation='h', 
    category_orders={
        'satisfaction-level': ["Sehr unzufrieden", "Unzufrieden", "Neutral", "Zufrieden", "Sehr zufrieden"],
        'task': [
            'Liste', 
            'Chord', 
            'Map', 
        ]
    },
)
fig.update_layout(
    title=chartLabel,
    yaxis_title='Aufgabe',
    xaxis_title='Empfundene Zufriedenheit',
    height=850,
    # margin=dict(
    #     pad=40
    # ),
    legend_title="Wie zufrieden war ich",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=24,
        # color="RebeccaPurple"
    )
)

fig.show()

# Tabellen
# Liste
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Wie zufrieden war ich?</b>', f"<b>Anzahl</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
        ["Sehr unzufrieden", "Unzufrieden", "Neutral", "Zufrieden", "Sehr zufrieden"],
        [
            0,
            dfList[dfList['satisfaction-level'] == 'Unzufrieden']['count'].values[0],
            dfList[dfList['satisfaction-level'] == 'Neutral']['count'].values[0],
            dfList[dfList['satisfaction-level'] == 'Zufrieden']['count'].values[0],
            dfList[dfList['satisfaction-level'] == 'Sehr zufrieden']['count'].values[0],
        ]
    ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title="Leichtigkeit eine Entscheidung zu Treffen: Liste",
    yaxis_title="Aufgabe",
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

# Chord
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Wie zufrieden war ich?</b>', f"<b>Anzahl</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
        ["Sehr unzufrieden", "Unzufrieden", "Neutral", "Zufrieden", "Sehr zufrieden"],
        [
            dfChord[dfChord['satisfaction-level'] == 'Sehr unzufrieden']['count'].values[0],
            dfChord[dfChord['satisfaction-level'] == 'Unzufrieden']['count'].values[0],
            dfChord[dfChord['satisfaction-level'] == 'Neutral']['count'].values[0],
            dfChord[dfChord['satisfaction-level'] == 'Zufrieden']['count'].values[0],
            dfChord[dfChord['satisfaction-level'] == 'Sehr zufrieden']['count'].values[0],
        ]
    ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title="Leichtigkeit eine Entscheidung zu Treffen: Chord",
    yaxis_title="Aufgabe",
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

# Chord
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Wie zufrieden war ich?</b>', f"<b>Anzahl</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
        ["Sehr unzufrieden", "Unzufrieden", "Neutral", "Zufrieden", "Sehr zufrieden"],
        [
            dfMap[dfMap['satisfaction-level'] == 'Sehr unzufrieden']['count'].values[0],
            dfMap[dfMap['satisfaction-level'] == 'Unzufrieden']['count'].values[0],
            dfMap[dfMap['satisfaction-level'] == 'Neutral']['count'].values[0],
            dfMap[dfMap['satisfaction-level'] == 'Zufrieden']['count'].values[0],
            dfMap[dfMap['satisfaction-level'] == 'Sehr zufrieden']['count'].values[0],
        ]
    ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title="Leichtigkeit eine Entscheidung zu Treffen: : Map",
    yaxis_title="Aufgabe",
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

## Quantitative Data

### List

In [28]:
filterArray = [
  109,
  113,
  124,
  130,
  138,
  145,
  157,
  158,
  191,
]
timeCleanedData = surveyData.drop(filterArray)

# List move
dfList = timeCleanedData[['list-m-move', 'list-m-click', 'list-aoi-pois']]
listMoveData = pd.DataFrame(dfList.loc[:, ('list-m-move')])
listMoveData['move-counts'] = listMoveData.apply(lambda row: len(json.loads(row['list-m-move'])), axis=1)

# List click
listClickData = pd.DataFrame(dfList.loc[:, ('list-m-click')])
listClickData['click-counts'] = listClickData.apply(lambda row: len(json.loads(row['list-m-click'])), axis=1)

# List AOI
listAoiDataCounts = pd.DataFrame()
listAoiData = pd.DataFrame(dfList.loc[:, ('list-aoi-pois')])

for row in listAoiData['list-aoi-pois']:
    singleListAoiCounts = pd.DataFrame(json.loads(row), columns = ['poi'])
    singleListAoiCounts = singleListAoiCounts.groupby(['poi'], dropna=False)['poi'].count().reset_index(name='count')
    data = pd.DataFrame({
        'Die Zwiebel': 0.0, 
        'Vapiano': 0.0,
        'Oishii': 0.0, 
        'American Diner Durlach': 0.0
    }, index=[0])

    if( 'Die Zwiebel' in singleListAoiCounts['poi'].values ): data['Die Zwiebel'] = singleListAoiCounts[singleListAoiCounts['poi'] == 'Die Zwiebel']['count'].values[0]
    if( 'Vapiano' in singleListAoiCounts['poi'].values ): data['Vapiano'] = singleListAoiCounts[singleListAoiCounts['poi'] == 'Vapiano']['count'].values[0]
    if( 'Oishii' in singleListAoiCounts['poi'].values ): data['Oishii'] = singleListAoiCounts[singleListAoiCounts['poi'] == 'Oishii']['count'].values[0]
    if( 'American Diner Durlach' in singleListAoiCounts['poi'].values ): data['American Diner Durlach'] = singleListAoiCounts[singleListAoiCounts['poi'] == 'American Diner Durlach']['count'].values[0]
    listAoiDataCounts = pd.concat([listAoiDataCounts, data], ignore_index=True)

categoryGroups = ['Mouse-Move-Events', 'Mouse-Click-Events', 'Die Zwiebel', 'Vapiano', 'Oishii', 'American Diner Durlach']
yLabel = "Anzahl an Mouse-Interaktionen"

# List mouse move
# Label strings
chartLabel = f"Mouse-Move-Events Liste"
fig = go.Figure()

fig.add_trace(go.Box(y=listMoveData['move-counts'], name=categoryGroups[0],
                marker_color = urCdColors[0]))

fig.update_layout(
    title=chartLabel,
    xaxis_title="Art an Mouse-Interaktionen",
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    width=300,
    height=650,
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=18,
        # color="RebeccaPurple"
    )
)

fig.update_layout(title_text=chartLabel)
fig.update_yaxes(ticksuffix = " ")
fig.show()

# List mouse clicks
# Label strings
chartLabel = f"Mouse-Click-Events Liste"
fig = go.Figure()

fig.add_trace(go.Box(y=listClickData['click-counts'], name=categoryGroups[1],
                marker_color = urCdColors[1]))

fig.update_layout(
    title=chartLabel,
    xaxis_title="Art an Mouse-Interaktionen",
    yaxis_title=yLabel,
    width=300,
    height=650,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=18,
        # color="RebeccaPurple"
    )
)

fig.update_layout(title_text=chartLabel)
fig.update_yaxes(ticksuffix = " ")
fig.show()

# List AOI clicks
# Label strings
chartLabel = f"Mouse-Click-Events nach AOIs – Liste"
fig = go.Figure()

fig.add_trace(go.Box(y=listAoiDataCounts[categoryGroups[2]], name=categoryGroups[2],
                marker_color = urCdColors[2]))
fig.add_trace(go.Box(y=listAoiDataCounts[categoryGroups[3]], name=categoryGroups[3],
                marker_color = urCdColors[3]))
fig.add_trace(go.Box(y=listAoiDataCounts[categoryGroups[4]], name=categoryGroups[4],
                marker_color = urCdColors[4]))
fig.add_trace(go.Box(y=listAoiDataCounts[categoryGroups[5]], name=categoryGroups[5],
                marker_color = urCdColors[5]))

fig.update_layout(
    title=chartLabel,
    xaxis_title="Art an Mouse-Interaktionen",
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    height=650,
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=18,
        # color="RebeccaPurple"
    )
)

fig.update_layout(title_text=chartLabel)
fig.update_yaxes(ticksuffix = " ")
fig.show()

# Table mouse data list
fig = go.Figure(data=[go.Table(
  header=dict(
    values=[
        '<b>Kennwert</b>', 
        f"<b>{categoryGroups[0]}</b>", 
        f"<b>{categoryGroups[1]}</b>", 
        f"<b>AOI {categoryGroups[2]}</b>",  
        f"<b>AOI {categoryGroups[3]}</b>",
        f"<b>AOI {categoryGroups[4]}</b>",
        f"<b>AOI {categoryGroups[5]}</b>"
    ],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['<b>Maximum</b>', '<b>Oberes Quartil</b>', '<b>Median</b>', '<b>Unteres Quartil</b>', '<b>Minimum</b>'],
      [round(listMoveData['move-counts'].describe()['max']), round(listMoveData['move-counts'].describe()['75%']), round(listMoveData['move-counts'].describe()['50%']), round(listMoveData['move-counts'].describe()['25%']), round(listMoveData['move-counts'].describe()['min'])],
      [round(listClickData['click-counts'].describe()['max']), round(listClickData['click-counts'].describe()['75%']), round(listClickData['click-counts'].describe()['50%']), round(listClickData['click-counts'].describe()['25%']), round(listClickData['click-counts'].describe()['min'])],
      [round(listAoiDataCounts[categoryGroups[2]].describe()['max']), round(listAoiDataCounts[categoryGroups[2]].describe()['75%']), round(listAoiDataCounts[categoryGroups[2]].describe()['50%']), round(listAoiDataCounts[categoryGroups[2]].describe()['25%']), round(listAoiDataCounts[categoryGroups[2]].describe()['min'])],
      [round(listAoiDataCounts[categoryGroups[3]].describe()['max']), round(listAoiDataCounts[categoryGroups[3]].describe()['75%']), round(listAoiDataCounts[categoryGroups[3]].describe()['50%']), round(listAoiDataCounts[categoryGroups[3]].describe()['25%']), round(listAoiDataCounts[categoryGroups[3]].describe()['min'])],
      [round(listAoiDataCounts[categoryGroups[4]].describe()['max']), round(listAoiDataCounts[categoryGroups[4]].describe()['75%']), round(listAoiDataCounts[categoryGroups[4]].describe()['50%']), round(listAoiDataCounts[categoryGroups[4]].describe()['25%']), round(listAoiDataCounts[categoryGroups[4]].describe()['min'])],
      [round(listAoiDataCounts[categoryGroups[5]].describe()['max']), round(listAoiDataCounts[categoryGroups[5]].describe()['75%']), round(listAoiDataCounts[categoryGroups[5]].describe()['50%']), round(listAoiDataCounts[categoryGroups[5]].describe()['25%']), round(listAoiDataCounts[categoryGroups[5]].describe()['min'])]
    ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor]*6],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title='Mouse-Events nach Art – Liste',
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

### Chord

In [29]:
filterArray = [
  109,
  113,
  124,
  130,
  138,
  145,
  157,
  158,
  191,
]
timeCleanedData = surveyData.drop(filterArray)

# Chord move
dfChord = timeCleanedData[['chord-m-move', 'chord-m-click', 'chord-aoi-pois', 'chord-aoi-choosen-components', 'chord-aoi-other-components']]
chordMoveData = pd.DataFrame(dfChord.loc[:, ('chord-m-move')])
chordMoveData['move-counts'] = chordMoveData.apply(lambda row: len(json.loads(row['chord-m-move'])), axis=1)

# Chord click
chordClickData = pd.DataFrame(dfChord.loc[:, ('chord-m-click')])
chordClickData['click-counts'] = chordClickData.apply(lambda row: len(json.loads(row['chord-m-click'])), axis=1)

# Chord AOI
chordAoiDataCounts = pd.DataFrame()
chordAoiData = pd.DataFrame(dfChord.loc[:, ('chord-aoi-pois', 'chord-aoi-choosen-components', 'chord-aoi-other-components')])

# Create counts for aoi poi events
poisRowList = []

for row in chordAoiData['chord-aoi-pois']:
    singleChordAoiPoisCounts = len(pd.DataFrame(json.loads(row), columns = ['name']))
    poisRowList.append(singleChordAoiPoisCounts)
chordAoiDataCounts['pois-counts'] = poisRowList

# Create counts for aoi choosen-components events
choosenComponentsRowList = []

for row in chordAoiData['chord-aoi-choosen-components']:
    singleChordAoiChoosenComponentsCounts = len(pd.DataFrame(json.loads(row), columns = ['name']))
    choosenComponentsRowList.append(singleChordAoiChoosenComponentsCounts)
chordAoiDataCounts['choosen-components-counts'] = choosenComponentsRowList

# Create counts for aoi choosen-components events
otherComponentsRowList = []

for row in chordAoiData['chord-aoi-other-components']:
    singleChordAoiChoosenComponentsCounts = len(pd.DataFrame(json.loads(row), columns = ['name']))
    otherComponentsRowList.append(singleChordAoiChoosenComponentsCounts)
chordAoiDataCounts['other-components-counts'] = otherComponentsRowList

categoryGroups = [
    'Mouse-Move-Events', 
    'Mouse-Click-Events', 
    'Mouse-Over-Events bei POIs', 
    'Mouse-Over-Events bei gewählten Komponenten', 
    'Mouse-Over-Events bei übrigen Komponenten'
]
yLabel = "Anzahl an Mouse-Interaktionen"

# Chord mouse move
# Label strings
chartLabel = f"Mouse-Move-Events Chord"
fig = go.Figure()

fig.add_trace(go.Box(y=chordMoveData['move-counts'], name=categoryGroups[0],
                marker_color = urCdColors[0]))

fig.update_layout(
    title=chartLabel,
    xaxis_title="Art an Mouse-Interaktionen",
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    width=300,
    height=650,
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=18,
        # color="RebeccaPurple"
    )
)

fig.update_layout(title_text=chartLabel)
fig.update_yaxes(ticksuffix = " ")
fig.show()

# Chord mouse clicks
# Label strings
chartLabel = f"Mouse-Click-Events Chord"
fig = go.Figure()

fig.add_trace(go.Box(y=chordClickData['click-counts'], name=categoryGroups[1],
                marker_color = urCdColors[1]))

fig.update_layout(
    title=chartLabel,
    xaxis_title="Art an Mouse-Interaktionen",
    yaxis_title=yLabel,
    width=300,
    height=650,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=18,
        # color="RebeccaPurple"
    )
)

fig.update_layout(title_text=chartLabel)
fig.update_yaxes(ticksuffix = " ")
fig.show()

# Chord AOI clicks
# Label strings
chartLabel = f"Mouse-Move-Events nach AOIs – Chord"
fig = go.Figure()

fig.add_trace(go.Box(y=chordAoiDataCounts['pois-counts'], name=categoryGroups[2],
                marker_color = urCdColors[2]))
fig.add_trace(go.Box(y=chordAoiDataCounts['choosen-components-counts'], name=categoryGroups[3],
                marker_color = urCdColors[3]))
fig.add_trace(go.Box(y=chordAoiDataCounts['other-components-counts'], name=categoryGroups[4],
                marker_color = urCdColors[4]))

fig.update_layout(
    title=chartLabel,
    xaxis_title="Art an Mouse-Interaktionen",
    yaxis_title=yLabel,
    height=850,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=18,
        # color="RebeccaPurple"
    )
)

fig.update_layout(title_text=chartLabel)
fig.update_yaxes(ticksuffix = " ")
fig.show()

# Table mouse data chord
fig = go.Figure(data=[go.Table(
  header=dict(
    values=[
        '<b>Kennwert</b>', 
        f"<b>{categoryGroups[0]}</b>", 
        f"<b>{categoryGroups[1]}</b>", 
        f"<b>AOI {categoryGroups[2]}</b>",  
        f"<b>AOI {categoryGroups[3]}</b>",
        f"<b>AOI {categoryGroups[4]}</b>",
    ],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['<b>Maximum</b>', '<b>Oberes Quartil</b>', '<b>Median</b>', '<b>Unteres Quartil</b>', '<b>Minimum</b>'],
      [round(chordMoveData['move-counts'].describe()['max']), round(chordMoveData['move-counts'].describe()['75%']), round(chordMoveData['move-counts'].describe()['50%']), round(chordMoveData['move-counts'].describe()['25%']), round(chordMoveData['move-counts'].describe()['min'])],
      [round(chordClickData['click-counts'].describe()['max']), round(chordClickData['click-counts'].describe()['75%']), round(chordClickData['click-counts'].describe()['50%']), round(chordClickData['click-counts'].describe()['25%']), round(chordClickData['click-counts'].describe()['min'])],
      [round(chordAoiDataCounts['pois-counts'].describe()['max']), round(chordAoiDataCounts['pois-counts'].describe()['75%']), round(chordAoiDataCounts['pois-counts'].describe()['50%']), round(chordAoiDataCounts['pois-counts'].describe()['25%']), round(chordAoiDataCounts['pois-counts'].describe()['min'])],
      [round(chordAoiDataCounts['choosen-components-counts'].describe()['max']), round(chordAoiDataCounts['choosen-components-counts'].describe()['75%']), round(chordAoiDataCounts['choosen-components-counts'].describe()['50%']), round(chordAoiDataCounts['choosen-components-counts'].describe()['25%']), round(chordAoiDataCounts['choosen-components-counts'].describe()['min'])],
      [round(chordAoiDataCounts['other-components-counts'].describe()['max']), round(chordAoiDataCounts['other-components-counts'].describe()['75%']), round(chordAoiDataCounts['other-components-counts'].describe()['50%']), round(chordAoiDataCounts['other-components-counts'].describe()['25%']), round(chordAoiDataCounts['other-components-counts'].describe()['min'])],
    ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor]*6],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title='Mouse-Events nach Art – Chord',
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

### Map

In [30]:
filterArray = [
  109,
  113,
  124,
  130,
  138,
  145,
  157,
  158,
  191,
]
timeCleanedData = surveyData.drop(filterArray)
# Map move
dfMap = timeCleanedData[['map-m-move', 'map-m-click', 'map-aoi-components-choice', 'map-aoi-pois', 'map-aoi-drag-map', 'map-aoi-drag-popup', 'map-aoi-zoom']]
mapMoveData = pd.DataFrame(dfMap.loc[:, ('map-m-move')])
mapMoveData['move-counts'] = mapMoveData.apply(lambda row: len(json.loads(row['map-m-move'])), axis=1)

# Map click
mapClickData = pd.DataFrame(dfMap.loc[:, ('map-m-click')])
mapClickData['click-counts'] = mapClickData.apply(lambda row: len(json.loads(row['map-m-click'])), axis=1)

# Map AOI
mapAoiDataCounts = pd.DataFrame()
mapAoiData = pd.DataFrame(dfMap.loc[:, ('map-aoi-components-choice', 'map-aoi-pois', 'map-aoi-drag-map', 'map-aoi-drag-popup', 'map-aoi-zoom')])

# Create counts for aoi poi events
componentsChoiceRowList = []

for row in mapAoiData['map-aoi-components-choice']:
    singleMapAoiComponentsChoiceCounts = len(pd.DataFrame(json.loads(row), columns = ['componentType']))
    componentsChoiceRowList.append(singleMapAoiComponentsChoiceCounts)
mapAoiDataCounts['components-choice-counts'] = componentsChoiceRowList

# Create counts for aoi poi events
poisRowList = []

for row in mapAoiData['map-aoi-pois']:
    singleMapAoiPoisCounts = len(pd.DataFrame(json.loads(row), columns = ['poi']))
    poisRowList.append(singleMapAoiPoisCounts)
mapAoiDataCounts['pois-counts'] = poisRowList

# Create counts for aoi drag-map events
# NOTE: Value is actually half, because eventStart and eventEnd was tracked
dragMapRowList = []

for row in mapAoiData['map-aoi-drag-map']:
    singleMapAoiDragMapCounts = len(pd.DataFrame(json.loads(row), columns = ['event']))
    dragMapRowList.append(singleMapAoiDragMapCounts)
mapAoiDataCounts['drag-map-counts'] = dragMapRowList

# Create counts for aoi drag-map events
# NOTE: Value is actually half, because eventStart and eventEnd was tracked
dragPopupRowList = []

for row in mapAoiData['map-aoi-drag-popup']:
    singleMapAoiDragPopupCounts = len(pd.DataFrame(json.loads(row), columns = ['event']))
    dragPopupRowList.append(singleMapAoiDragPopupCounts)
mapAoiDataCounts['drag-popup-counts'] = dragPopupRowList

# Create counts for aoi zoom events
zoomRowList = []

for row in mapAoiData['map-aoi-zoom']:
    singleMapAoiZoomCounts = len(pd.DataFrame(json.loads(row), columns = ['event']))
    zoomRowList.append(singleMapAoiZoomCounts)
mapAoiDataCounts['zoom-counts'] = zoomRowList

categoryGroups = [
    'Mouse-Move-Events', 
    'Mouse-Click-Events', 
    'Mouse-Click-Events bei Ansichtswechsel', 
    'Mouse-Click-Events bei POIs', 
    'Mouse-Click-Events bei Map Drag\'n\'Drop', 
    'Mouse-Click-Events bei Popup Drag\'n\'Drop',
    'Zoom-Events Mouse und Schaltfläche'
]
yLabel = "Anzahl an Interaktionen"

# Map mouse move
# Label strings
chartLabel = f"Mouse-Move-Events Map"
fig = go.Figure()

fig.add_trace(go.Box(y=mapMoveData['move-counts'], name=categoryGroups[0],
                marker_color = urCdColors[0]))

fig.update_layout(
    title=chartLabel,
    xaxis_title="Art an Mouse-Interaktionen",
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    width=300,
    height=650,
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=18,
        # color="RebeccaPurple"
    )
)

fig.update_layout(title_text=chartLabel)
fig.update_yaxes(ticksuffix = " ")
fig.show()

# Map mouse clicks
# Label strings
chartLabel = f"Mouse-Click-Events Map"
fig = go.Figure()

fig.add_trace(go.Box(y=mapClickData['click-counts'], name=categoryGroups[1],
                marker_color = urCdColors[1]))

fig.update_layout(
    title=chartLabel,
    xaxis_title="Art an Mouse-Interaktionen",
    yaxis_title=yLabel,
    width=300,
    height=650,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=18,
        # color="RebeccaPurple"
    )
)

fig.update_layout(title_text=chartLabel)
fig.update_yaxes(ticksuffix = " ")
fig.show()

# Map AOI clicks
# Label strings
chartLabel = f"Mouse-Events nach AOIs – Map"
fig = go.Figure()

fig.add_trace(go.Box(y=mapAoiDataCounts['components-choice-counts'], name=categoryGroups[2],
                marker_color = urCdColors[2]))
fig.add_trace(go.Box(y=mapAoiDataCounts['pois-counts'], name=categoryGroups[3],
                marker_color = urCdColors[3]))
fig.add_trace(go.Box(y=mapAoiDataCounts['drag-map-counts'], name=categoryGroups[4],
                marker_color = urCdColors[4]))
fig.add_trace(go.Box(y=mapAoiDataCounts['drag-popup-counts'], name=categoryGroups[5],
                marker_color = urCdColors[5]))
fig.add_trace(go.Box(y=mapAoiDataCounts['zoom-counts'], name=categoryGroups[6],
                marker_color = urCdColors[6]))

fig.update_layout(
    title=chartLabel,
    xaxis_title="Art an Mouse-Interaktionen",
    yaxis_title=yLabel,
    height=850,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=18,
        # color="RebeccaPurple"
    )
)

fig.update_layout(title_text=chartLabel)
fig.update_yaxes(ticksuffix = " ")
fig.show()

# Table mouse data map
fig = go.Figure(data=[go.Table(
  header=dict(
    values=[
        '<b>Kennwert</b>', 
        f"<b>{categoryGroups[0]}</b>", 
        f"<b>{categoryGroups[1]}</b>", 
        f"<b>AOI {categoryGroups[2]}</b>",  
        f"<b>AOI {categoryGroups[3]}</b>",
        f"<b>AOI {categoryGroups[4]}</b>",
        f"<b>AOI {categoryGroups[5]}</b>",
        f"<b>AOI {categoryGroups[6]}</b>",
    ],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      ['<b>Maximum</b>', '<b>Oberes Quartil</b>', '<b>Median</b>', '<b>Unteres Quartil</b>', '<b>Minimum</b>'],
      [round(mapMoveData['move-counts'].describe()['max']), round(mapMoveData['move-counts'].describe()['75%']), round(mapMoveData['move-counts'].describe()['50%']), round(mapMoveData['move-counts'].describe()['25%']), round(mapMoveData['move-counts'].describe()['min'])],
      [round(mapClickData['click-counts'].describe()['max']), round(mapClickData['click-counts'].describe()['75%']), round(mapClickData['click-counts'].describe()['50%']), round(mapClickData['click-counts'].describe()['25%']), round(mapClickData['click-counts'].describe()['min'])],
      [round(mapAoiDataCounts['components-choice-counts'].describe()['max']), round(mapAoiDataCounts['components-choice-counts'].describe()['75%']), round(mapAoiDataCounts['components-choice-counts'].describe()['50%']), round(mapAoiDataCounts['components-choice-counts'].describe()['25%']), round(mapAoiDataCounts['components-choice-counts'].describe()['min'])],
      [round(mapAoiDataCounts['pois-counts'].describe()['max']), round(mapAoiDataCounts['pois-counts'].describe()['75%']), round(mapAoiDataCounts['pois-counts'].describe()['50%']), round(mapAoiDataCounts['pois-counts'].describe()['25%']), round(mapAoiDataCounts['pois-counts'].describe()['min'])],
      [round(mapAoiDataCounts['drag-map-counts'].describe()['max']), round(mapAoiDataCounts['drag-map-counts'].describe()['75%']), round(mapAoiDataCounts['drag-map-counts'].describe()['50%']), round(mapAoiDataCounts['drag-map-counts'].describe()['25%']), round(mapAoiDataCounts['drag-map-counts'].describe()['min'])],
      [round(mapAoiDataCounts['drag-popup-counts'].describe()['max']), round(mapAoiDataCounts['drag-popup-counts'].describe()['75%']), round(mapAoiDataCounts['drag-popup-counts'].describe()['50%']), round(mapAoiDataCounts['drag-popup-counts'].describe()['25%']), round(mapAoiDataCounts['drag-popup-counts'].describe()['min'])],
      [round(mapAoiDataCounts['zoom-counts'].describe()['max']), round(mapAoiDataCounts['zoom-counts'].describe()['75%']), round(mapAoiDataCounts['zoom-counts'].describe()['50%']), round(mapAoiDataCounts['zoom-counts'].describe()['25%']), round(mapAoiDataCounts['zoom-counts'].describe()['min'])],
    ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor]*6],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title='Mouse-Events nach Art –Map',
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

## System Usability Scale

### SUS Bar Chart

In [31]:
filterArray = [
  109,
  113,
  124,
  130,
  138,
  145,
  157,
  158,
  191,
]
timeCleanedData = surveyData.drop(filterArray)

df = timeCleanedData[[ "sus-01", "sus-02", "sus-03", "sus-04", "sus-05", "sus-06", "sus-07", "sus-08", "sus-09", "sus-10"]]
susDf = df.replace(['Stimme überhaupt nicht zu – 1', '2', '3', '4', 'Stimme voll zu – 5'], [1, 2, 3, 4, 5])
df = susDf.apply(pd.Series.value_counts)
print(len(timeCleanedData))

categoryGroups = [
    'Stimme überhaupt nicht zu', 
    'Stimme nicht zu', 
    'Neutral', 
    'Stimme zu', 
    'Stimme voll zu', 
]

otherQuestions = [
    '1. Ich denke, dass ich die Chord- und Map-Ansicht gerne nochmals benutzen würde.',
    '2. Ich fand die Chord- und Map-Ansicht unnötig komplex.',
    '3. Ich fand die Chord- und Map-Ansicht einfach zu benutzen.',
    '4. Ich glaube, ich könnte Hilfe benötigen, um die Chord- und Map-Ansicht besser nutzen zu können.',
    '5. Ich fand, dass die verschiedenen Funktionen gut in die Chord- und Map-Ansicht eingebaut waren.',
    '6. Ich denke, die Chord- und Map-Ansicht enthalten zu viele Ungereimtheiten.',
    '7. Ich glaube, dass den meisten Menschen die Bedienung der Chord- und Map-Ansicht leicht fällt.',
    '8. Ich fand die Chord- und Map-Ansicht sehr umständlich zu benutzen.',
    '9. Ich fühlte mich bei der Benutzung der Chord- und Map-Ansicht sehr sicher.',
    '10. Ich musste viel ausprobieren, bevor ich anfangen konnte, die Chord- und Map-Ansicht richtig zu verwenden.'
]


# Label strings
chartLabel = f"System Usability Scale, n={len(timeCleanedData[['quantitatively-questions-show-all-criteria']])}"

fig = go.Figure()

stackedLables = pd.DataFrame({'counts': df.iloc[0]})
stackedLables['text'] =  stackedLables.apply(lambda row: "" + str(round(row['counts']/len(timeCleanedData[['quantitatively-questions-show-all-criteria']])*100, 1)) + " % (" + str(row['counts']) + ")", axis=1)
fig.add_trace(go.Bar(
    y=otherQuestions, 
    x=df.iloc[0].values,
    orientation='h',
    name=categoryGroups[0],
    marker=dict(
        color=urCdColors[0]
    ),
    text=stackedLables['text'].values,
))

stackedLables = pd.DataFrame({'counts': df.iloc[1]})
stackedLables['text'] =  stackedLables.apply(lambda row: "" + str(round(row['counts']/len(timeCleanedData[['quantitatively-questions-show-all-criteria']])*100, 1)) + " % (" + str(row['counts']) + ")", axis=1)
fig.add_trace(go.Bar(
    y=otherQuestions, 
    x=df.iloc[1].values,
    orientation='h',
    name=categoryGroups[1],
    marker=dict(
        color=urCdColors[1] 
    ),
    text=stackedLables['text'].values,
))

stackedLables = pd.DataFrame({'counts': df.iloc[2]})
stackedLables['text'] =  stackedLables.apply(lambda row: "" + str(round(row['counts']/len(timeCleanedData[['quantitatively-questions-show-all-criteria']])*100, 1)) + " % (" + str(row['counts']) + ")", axis=1)
fig.add_trace(go.Bar(
    y=otherQuestions, 
    x=df.iloc[2].values,
    orientation='h',
    name=categoryGroups[2],
    marker=dict(
        color=urCdColors[2]
    ),
    text=stackedLables['text'].values,
))

stackedLables = pd.DataFrame({'counts': df.iloc[3]})
stackedLables['text'] =  stackedLables.apply(lambda row: "" + str(round(row['counts']/len(timeCleanedData[['quantitatively-questions-show-all-criteria']])*100, 1)) + " % (" + str(row['counts']) + ")", axis=1)
fig.add_trace(go.Bar(
    y=otherQuestions, 
    x=df.iloc[3].values,
    orientation='h',
    name=categoryGroups[3],
    marker=dict(
        color=urCdColors[3]
    ),
    text=stackedLables['text'].values,
))

stackedLables = pd.DataFrame({'counts': df.iloc[4]})
stackedLables['text'] =  stackedLables.apply(lambda row: "" + str(round(row['counts']/len(timeCleanedData[['quantitatively-questions-show-all-criteria']])*100, 1)) + " % (" + str(row['counts']) + ")", axis=1)
fig.add_trace(go.Bar(
    y=otherQuestions, 
    x=df.iloc[4].values,
    orientation='h',
    name=categoryGroups[4],
    marker=dict(
        color=urCdColors[4]
    ),
    text=stackedLables['text'].values,
))
fig.update_yaxes(ticksuffix = "   ")

fig.update_layout(
    title=chartLabel,
    yaxis_title='Frage',
    yaxis_autorange="reversed",
    xaxis_title='Zustimmung nach 5-Fach-Likert',
    barmode='stack',
    width=1920,
    height=1080,
    # margin=dict(
    #     pad=40
    # ),
    legend_title="Zustimmung in Stufen",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=15,
        # color="RebeccaPurple"
    )
)

fig.show()

194


### SUS Table

In [32]:
categoryGroups = [
    'Stimme überhaupt nicht zu', 
    'Stimme nicht zu', 
    'Neutral', 
    'Stimme zu', 
    'Stimme voll zu', 
]

otherQuestions = [
    '1. Ich denke, dass ich die Chord- und Map-Ansicht gerne nochmals benutzen würde.',
    '2. Ich fand die Chord- und Map-Ansicht unnötig komplex.',
    '3. Ich fand die Chord- und Map-Ansicht einfach zu benutzen.',
    '4. Ich glaube, ich könnte Hilfe benötigen, um die Chord- und Map-Ansicht besser nutzen zu können.',
    '5. Ich fand, dass die verschiedenen Funktionen gut in die Chord- und Map-Ansicht eingebaut waren.',
    '6. Ich denke, die Chord- und Map-Ansicht enthalten zu viele Ungereimtheiten.',
    '7. Ich glaube, dass den meisten Menschen die Bedienung der Chord- und Map-Ansicht leicht fällt.',
    '8. Ich fand die Chord- und Map-Ansicht sehr umständlich zu benutzen.',
    '9. Ich fühlte mich bei der Benutzung der Chord- und Map-Ansicht sehr sicher.',
    '10. Ich musste viel ausprobieren, bevor ich anfangen konnte, die Chord- und Map-Ansicht richtig zu verwenden.'
]

fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>SUS Frage</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>", f"<b>{categoryGroups[3]}</b>", f"<b>{categoryGroups[4]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      [
        f"<b>{otherQuestions[0]}</b>",
        f"<b>{otherQuestions[1]}</b>",
        f"<b>{otherQuestions[2]}</b>",
        f"<b>{otherQuestions[3]}</b>",
        f"<b>{otherQuestions[4]}</b>",
        f"<b>{otherQuestions[5]}</b>",
        f"<b>{otherQuestions[6]}</b>",
        f"<b>{otherQuestions[7]}</b>",
        f"<b>{otherQuestions[8]}</b>",
        f"<b>{otherQuestions[9]}</b>",
      ],
      df.iloc[0].to_numpy(),
      df.iloc[1].to_numpy(),
      df.iloc[2].to_numpy(),
      df.iloc[3].to_numpy(),
      df.iloc[4].to_numpy(),
    ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor]*6],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    height=700,
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()


### SUS Score Plots

In [33]:
filterArray = [
  109,
  113,
  124,
  130,
  138,
  145,
  157,
  158,
  191,
]
timeCleanedData = surveyData.drop(filterArray)
df = timeCleanedData[[ "sus-01", "sus-02", "sus-03", "sus-04", "sus-05", "sus-06", "sus-07", "sus-08", "sus-09", "sus-10", "id", "duration-over-all", "duration-group-list-task", "duration-group-chord-task", "duration-group-map-task", "list-question-number", "chord-question-number", "map-question-number", "highest-preferenced-viz"]]
# df = timeCleanedData[[ "id", "duration-over-all", "duration-group-list-task", "duration-group-chord-task", "duration-group-map-task", "list-question-number", "chord-question-number", "map-question-number", "highest-preferenced-viz"]]
susDf = df.replace(['Stimme überhaupt nicht zu – 1', '2', '3', '4', 'Stimme voll zu – 5'], [1, 2, 3, 4, 5])

susDf['sus-score'] = 2.5 * (
  (susDf['sus-01'] - 1) + 
  (5 - susDf['sus-02']) +
  (susDf['sus-03'] -1) +
  (5 - susDf['sus-04']) + 
  (susDf['sus-05'] -1) + 
  (5 - susDf['sus-06']) + 
  (susDf['sus-07'] -1) + 
  (5 - susDf['sus-08']) + 
  (susDf['sus-09'] -1) + 
  (5 - susDf['sus-10'])
) 

dfAll = susDf['sus-score']
dfList = susDf[susDf['highest-preferenced-viz'] == 'Liste']['sus-score']
dfChord = susDf[susDf['highest-preferenced-viz'] == 'Chord']['sus-score']
dfMap = susDf[susDf['highest-preferenced-viz'] == 'Map']['sus-score']

# Label strings
# chartLabel = f"Dauer der Aufgaben, n={len(susDf)}"
chartLabel = f"SUS-Score im Vergleich favorisierter Ansichten"
categoryGroups = [f"Gesamt, n={len(dfAll)}", f"Liste favorisiert, n={len(dfList)}", f"Chord favorisiert, n={len(dfChord)}", f"Karte favorisiert, n={len(dfMap)}"]
yLabel = "SUS-Score"
fig = go.Figure()

# All SUS-Scores 
fig.add_trace(go.Box(y=dfAll, name=categoryGroups[0],
                marker_color = urCdColors[0]))
# List SUS-Scores
fig.add_trace(go.Box(y=dfList, name=categoryGroups[1],
                marker_color = urCdColors[1]))
# Chord SUS-Scores
fig.add_trace(go.Box(y=dfChord, name=categoryGroups[2],
                marker_color = urCdColors[2]))
# Map SUS-Scores 
fig.add_trace(go.Box(y=dfMap, name=categoryGroups[3],
                marker_color = urCdColors[3]))

# fig.update_traces(quartilemethod="exclusive")
fig.update_layout(
    title=chartLabel,
    xaxis_title="Favorisierte Ansicht",
    yaxis_title=yLabel,
    height=850,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=24,
        # color="RebeccaPurple"
    )
)

fig.update_layout(title_text=chartLabel)
fig.update_yaxes(ticksuffix = " ")

fig.show()

### SUS Data

In [34]:
# SUS CSV output

susAll = susDf[["sus-01", "sus-02", "sus-03", "sus-04", "sus-05", "sus-06", "sus-07", "sus-08", "sus-09", "sus-10"]]

dfList = susDf[susDf['highest-preferenced-viz'] == 'Liste'][["sus-01", "sus-02", "sus-03", "sus-04", "sus-05", "sus-06", "sus-07", "sus-08", "sus-09", "sus-10"]]
dfChord = susDf[susDf['highest-preferenced-viz'] == 'Chord'][["sus-01", "sus-02", "sus-03", "sus-04", "sus-05", "sus-06", "sus-07", "sus-08", "sus-09", "sus-10"]]
dfMap = susDf[susDf['highest-preferenced-viz'] == 'Map'][["sus-01", "sus-02", "sus-03", "sus-04", "sus-05", "sus-06", "sus-07", "sus-08", "sus-09", "sus-10"]]

susAll.to_csv('sus-data.csv', index=False, sep=';')
dfList.to_csv('sus-data-list-preferred.csv', index=False, sep=';')
dfChord.to_csv('sus-data-chord-preferred.csv', index=False, sep=';')
dfMap.to_csv('sus-data-map-preferred.csv', index=False, sep=';')


Basierend auf dem Auswertungstemplate des SUS auf Deutscht von Bernard Rummel wurden folgende Werte berechnet 

Vgl. https://blogs.sap.com/2016/02/01/system-usability-scale-jetzt-auch-auf-deutsch/

Rummel, Berndard: »System Usability Scale – jetzt auch auf Deutsch.«, https://blogs.sap.com/2016/02/01/system-usability-scale-jetzt-auch-auf-deutsch/, 2016, aufgerufen am: 11.08.21.


## SUS ALL Score 56,9
n (all) = 194 

| Total SUS Score | 56,9 |
|---|---|
| Std.Dev | 19,4 |
| 95% Confidence Interval | 2,729510488 |
| Upper CI Bound | 58,3 |
| Lower CI Bound | 55,6 |

## SUS List Score 42,4
n (List Preferred) = 50 

| Total SUS Score | 42,4 |
|---|---|
| Std.Dev | 16,9 |
| 95% Confidence Interval | 4,687217572 |
| Upper CI Bound | 44,7 |
| Lower CI Bound | 40,1 |

## SUS Chord Score 66,2
n (Chord Preferred) = 58 

| Total SUS Score | 66,2 |
|---|---|
| Std.Dev | 18,1 |
| 95% Confidence Interval | 4,657111984 |
| Upper CI Bound | 68,5 |
| Lower CI Bound | 63,9 |

## SUS Map Score 59,1
n (Map Preferred) = 86 

| Total SUS Score | 59,1 |
|---|---|
| Std.Dev | 16,8 |
| 95% Confidence Interval | 3,550357024 |
| Upper CI Bound | 60,9 |
| Lower CI Bound | 57,3 |

Basiernd auf Jess Sauros Interprätationshilfe: vgl. https://measuringu.com/interpret-sus-score/

Ist der Wert OK. 

![SUS Sorce](https://measuringu.com/wp-content/uploads/2018/09/sus-scale-adj.jpg)


## Other Questions

In [43]:
df = surveyData[[
    "quantitatively-questions-show-all-criteria",
    "quantitatively-questions-good-impression-from-viz",
    "quantitatively-questions-show-full-info",
    "quantitatively-questions-more-responsibility",
    "quantitatively-questions-show-understandable",
    "quantitatively-questions-just-show-results-no-differences"
]]

susDf = df.replace(['Stimme überhaupt nicht zu – 1', '2', '3', '4', 'Stimme voll zu – 5'], [1, 2, 3, 4, 5])

filterArray = [
  109,
  113,
  124,
  130,
  138,
  145,
  157,
  158,
  191,
]
timeCleanedData = susDf.drop(filterArray)

df = timeCleanedData.apply(pd.Series.value_counts)

categoryGroups = [
    'Stimme überhaupt nicht zu', 
    'Stimme nicht zu', 
    'Neutral', 
    'Stimme zu', 
    'Stimme voll zu', 
]

# otherQuestions = [
#     'Glauben Sie, man sollte bei Vergleichsanwendungen auch die Kriterien kommunizieren, die nicht direkt bei der Filterung ausgewählt wurden?',
#     'Glauben Sie, Sie können sich mit einer der Ansichten einen passenden Eindruck von dem jeweiligen Ort verschaffen?',
#     'Glauben Sie, man sollte Nutzerinnen und Nutzern Informationen zu entsprechenden Suchen immer möglichst umfassend anzeigen?',
#     'Glauben Sie, es ist sinnvoll von den Nutzerinnen und Nutzern mehr Verantwortung während des Suchens und Vergleichens zu fordern?',
#     'Glauben Sie, es ist nötig, Ergebnisse von Suchen möglichst nachvollziehbar darzustellen?',
#     'Glauben Sie, es ist sinnvoll, wenn ein Algorithmus Ergebnisse darstellt ohne die Unterschiede spezifisch aufzuzeigen?',
# ]

otherQuestions = [
    '1. Übrige Kriterien kommunizieren',
    '2. Eindruck kann sich verschafft werden',
    '3. Möglichst umfassend Anzeigen von Ergebnissen',
    '4. Verantwortung fordern?',
    '5. Nachvollziehbare Darstellung?',
    '6. Darstellung ohne spezifisch Unterschiede',
]

# Label strings
chartLabel = f"Weitere qualitative Fragen, n={len(timeCleanedData[['quantitatively-questions-show-all-criteria']])}"

fig = go.Figure()

stackedLables = pd.DataFrame({'counts': df.iloc[0]})
stackedLables
stackedLables['text'] =  stackedLables.apply(lambda row: "" + str(round(row['counts']/len(timeCleanedData[['quantitatively-questions-show-all-criteria']])*100, 1)) + " % (" + str(row['counts']) + ")", axis=1)
fig.add_trace(go.Bar(
    y=otherQuestions, 
    x=df.iloc[0].values,
    orientation='h',
    name=categoryGroups[0],
    marker=dict(
        color=urCdColors[0]
    ),
    text=stackedLables['text'].values,
))

stackedLables = pd.DataFrame({'counts': df.iloc[1]})
stackedLables['text'] =  stackedLables.apply(lambda row: "" + str(round(row['counts']/len(timeCleanedData[['quantitatively-questions-show-all-criteria']])*100, 1)) + " % (" + str(row['counts']) + ")", axis=1)
fig.add_trace(go.Bar(
    y=otherQuestions, 
    x=df.iloc[1].values,
    orientation='h',
    name=categoryGroups[1],
    marker=dict(
        color=urCdColors[1] 
    ),
    text=stackedLables['text'].values,
))

stackedLables = pd.DataFrame({'counts': df.iloc[2]})
stackedLables['text'] =  stackedLables.apply(lambda row: "" + str(round(row['counts']/len(timeCleanedData[['quantitatively-questions-show-all-criteria']])*100, 1)) + " % (" + str(row['counts']) + ")", axis=1)
fig.add_trace(go.Bar(
    y=otherQuestions, 
    x=df.iloc[2].values,
    orientation='h',
    name=categoryGroups[2],
    marker=dict(
        color=urCdColors[2]
    ),
    text=stackedLables['text'].values,
))

stackedLables = pd.DataFrame({'counts': df.iloc[3]})
stackedLables['text'] =  stackedLables.apply(lambda row: "" + str(round(row['counts']/len(timeCleanedData[['quantitatively-questions-show-all-criteria']])*100, 1)) + " % (" + str(row['counts']) + ")", axis=1)
fig.add_trace(go.Bar(
    y=otherQuestions, 
    x=df.iloc[3].values,
    orientation='h',
    name=categoryGroups[3],
    marker=dict(
        color=urCdColors[3]
    ),
    text=stackedLables['text'].values,
))

stackedLables = pd.DataFrame({'counts': df.iloc[4]})
stackedLables['text'] =  stackedLables.apply(lambda row: "" + str(round(row['counts']/len(timeCleanedData[['quantitatively-questions-show-all-criteria']])*100, 1)) + " % (" + str(row['counts']) + ")", axis=1)
fig.add_trace(go.Bar(
    y=otherQuestions, 
    x=df.iloc[4].values,
    orientation='h',
    name=categoryGroups[4],
    marker=dict(
        color=urCdColors[4]
    ),
    text=stackedLables['text'].values,
))


fig.update_layout(
    title=chartLabel,
    yaxis_title='Frage',
    yaxis_autorange="reversed",
    xaxis_title='Zustimmung nach 5-Fach-Likert',
    barmode='stack',
    width=2160,
    height=1080,
    # margin=dict(
    #     pad=40
    # ),
    legend_title="Zustimmung in Stufen",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=18,
        # color="RebeccaPurple"
    )
)

fig.update_yaxes(ticksuffix = "  ")
fig.show()

### Other Questions Table

In [36]:
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>SUS Frage</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>", f"<b>{categoryGroups[3]}</b>", f"<b>{categoryGroups[4]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      [
        f"<b>{otherQuestions[0]}</b>",
        f"<b>{otherQuestions[1]}</b>",
        f"<b>{otherQuestions[2]}</b>",
        f"<b>{otherQuestions[3]}</b>",
        f"<b>{otherQuestions[4]}</b>",
        f"<b>{otherQuestions[5]}</b>",
      ],
      df.iloc[0].to_numpy(),
      df.iloc[1].to_numpy(),
      df.iloc[2].to_numpy(),
      df.iloc[3].to_numpy(),
      df.iloc[4].to_numpy(),
    ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor]*6],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    # yaxis_title=yLabel,
    # legend_title="Legend Title",
    height=700,
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

## Chord and Map over List

In [37]:
filterArray = [
  109,
  113,
  124,
  130,
  138,
  145,
  157,
  158,
  191,
]
timeCleanedData = surveyData.drop(filterArray)

dfAll = timeCleanedData[['preference-chord-or-map-over-list']]
dfAll = dfAll.groupby(['preference-chord-or-map-over-list'], dropna=False)['preference-chord-or-map-over-list'].count().reset_index(name='count')

# Label strings
chartLabel = f"Bevorzugte Visualisierung Chord oder Map im Vergleich zur Liste, n={len(timeCleanedData[['preference-chord-or-map-over-list']])}"
categoryGroups = ["Ja", "Nein"]

fig = make_subplots(rows=1, cols=1, 
    specs=[
        [{'type':'domain'}],
    ],
    # subplot_titles="Chord oder Map favorisiert"
)

fig.add_trace(go.Pie(labels=categoryGroups, values=dfAll['count'], sort=False, direction='clockwise', marker_colors=urCdColors),
              1, 1)

fig.update_traces(textinfo='percent+label+value')

fig.update_layout(
    title=chartLabel,
    title_font_size=18,
    yaxis_title=yLabel,
    width=850,
    height=850,
    #legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=24,
        # color="RebeccaPurple"
    )
)
fig.update(layout_showlegend=True)
fig.show()

# Table mood development
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Chord oder Map favorisiert</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      # [surveyData[surveyData['sex'] == 'männlich'], surveyData[surveyData['sex'] == 'nicht angegeben'], surveyData[surveyData['sex'] == 'weiblich']]],
        ['<b>Gesamt</b>'],
        [dfAll.iloc[0]['count']],
        [dfAll.iloc[1]['count']],
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 12)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

## Absolute Favorite Visualization

In [38]:
filterArray = [
  109,
  113,
  124,
  130,
  138,
  145,
  157,
  158,
  191,
]
timeCleanedData = surveyData.drop(filterArray)

dfAll = timeCleanedData[['highest-preferenced-viz', 'list-question-number', 'map-question-number', 'map-question-number']]
dfAll = dfAll.groupby(['highest-preferenced-viz'], dropna=False)['highest-preferenced-viz'].count().reset_index(name='count')
print(dfAll)

dfListe = timeCleanedData.loc[timeCleanedData['list-question-number'] == 6][['highest-preferenced-viz', 'list-question-number']]
dfListe = dfListe.groupby(['highest-preferenced-viz'], dropna=False)['highest-preferenced-viz'].count().reset_index(name='count')

dfChord = timeCleanedData.loc[timeCleanedData['chord-question-number'] == 6][['highest-preferenced-viz', 'chord-question-number']]
dfChord = dfChord.groupby(['highest-preferenced-viz'], dropna=False)['highest-preferenced-viz'].count().reset_index(name='count')

dfMap = timeCleanedData.loc[timeCleanedData['map-question-number'] == 6][['highest-preferenced-viz', 'map-question-number']]
dfMap = dfMap.groupby(['highest-preferenced-viz'], dropna=False)['highest-preferenced-viz'].count().reset_index(name='count')

# Label strings
chartLabel = f"Am meisten favorisierte Visualisierung nach Reihenfolge, n={len(timeCleanedData[['highest-preferenced-viz']])}"
categoryGroups = ["Liste", "Chord", "Map"]
fig = make_subplots(rows=2, cols=3, 
    specs=[
        [None, {'type':'domain'}, None],
        [{'type':'domain'}, {'type':'domain'}, {'type':'domain'}],
    ],
    subplot_titles=("Gesamt","Aufgabe 1: Liste ", "Aufgabe 1: Chord",  "Aufgabe 1: Map"))

fig.add_trace(go.Pie(labels=[categoryGroups[1], categoryGroups[0], categoryGroups[2]], values=dfAll['count'], sort=False, direction='clockwise', marker_colors=urCdColors),
              1, 2)
fig.add_trace(go.Pie(labels=[categoryGroups[1], categoryGroups[0], categoryGroups[2]], values=dfListe['count'], sort=False, direction='clockwise', marker_colors=urCdColors),
              2, 1)
fig.add_trace(go.Pie(labels=[categoryGroups[1], categoryGroups[0], categoryGroups[2]], values=dfChord['count'], sort=False, direction='clockwise', marker_colors=urCdColors),
              2, 2)
fig.add_trace(go.Pie(labels=[categoryGroups[1], categoryGroups[0], categoryGroups[2]], values=dfMap['count'], sort=False, direction='clockwise', marker_colors=urCdColors),
              2, 3)

fig.update_traces(textinfo='percent+label+value')
fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    height=850,
    #legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=18,
        # color="RebeccaPurple"
    )
)
fig.update(layout_showlegend=True)
fig.update_annotations(yshift=24)
fig.show()

# Table mood development
fig = go.Figure(data=[go.Table(
  header=dict(
    values=['<b>Favorisierte Visualisierung</b>', f"<b>{categoryGroups[0]}</b>", f"<b>{categoryGroups[1]}</b>", f"<b>{categoryGroups[2]}</b>"],
    line_color='darkslategray',
    fill_color=headerColor,
    align=['left','center'],
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=[
      # [surveyData[surveyData['sex'] == 'männlich'], surveyData[surveyData['sex'] == 'nicht angegeben'], surveyData[surveyData['sex'] == 'weiblich']]],
        ['<b>Gesamt</b>','<b>Liste zuerst</b>', '<b>Chord zuerst</b>', '<b>Map zuerst</b>'],
        [dfAll.iloc[1]['count'], dfListe.iloc[1]['count'], dfChord.iloc[1]['count'], dfMap.iloc[1]['count']],
        [dfAll.iloc[0]['count'], dfListe.iloc[0]['count'], dfChord.iloc[0]['count'], dfMap.iloc[0]['count']],
        [dfAll.iloc[2]['count'], dfListe.iloc[2]['count'], dfChord.iloc[2]['count'], dfMap.iloc[2]['count']],
      ],
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [[rowOddColor,rowEvenColor,rowOddColor, rowEvenColor,rowOddColor]*5],
    align = ['left', 'center'],
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title=chartLabel,
    yaxis_title=yLabel,
    # legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=12,
        # color="RebeccaPurple"
    )
)

fig.show()

  highest-preferenced-viz  count
0                   Chord     58
1                   Liste     50
2                     Map     86


## Qualitative Feedback

### Quantity of Feedbacks

In [39]:
feedbackComments = surveyData[[
  "id", 
  "preference-reason",
  "qualitative-questions-pros-cons-chord",
  "qualitative-questions-pros-cons-map",
  "qualitative-questions-pros-cons-list",
  "comments",
]]

pd.options.mode.chained_assignment = None  # default='warn'
feedbackComments.loc[feedbackComments['comments'].str.contains('versuchpersonenstunden', case=False, na=False), 'comments'] = np.NaN
feedbackComments.loc[feedbackComments['comments'].str.contains('versuchspersonenstunden', case=False, na=False), 'comments'] = np.NaN

replaceStrings = [
  '^-$',
  '^–$',
  '^...$',
  '^…$',
  '^/$',
  '^nichts',
  '^Nichts',
  '^k$',
  '^K$',
  '^keine$',
  '^Keine$',
]

# First Field Replacements
feedbackComments['preference-reason'] = feedbackComments['preference-reason'].replace({replaceStrings[0]:np.NaN}, regex=True)
feedbackComments['preference-reason'] = feedbackComments['preference-reason'].replace({replaceStrings[1]:np.NaN}, regex=True)
feedbackComments['preference-reason'] = feedbackComments['preference-reason'].replace({replaceStrings[2]:np.NaN}, regex=True)
feedbackComments['preference-reason'] = feedbackComments['preference-reason'].replace({replaceStrings[3]:np.NaN}, regex=True)
feedbackComments['preference-reason'] = feedbackComments['preference-reason'].replace({replaceStrings[4]:np.NaN}, regex=True)
feedbackComments['preference-reason'] = feedbackComments['preference-reason'].replace({replaceStrings[5]:np.NaN}, regex=True)
feedbackComments['preference-reason'] = feedbackComments['preference-reason'].replace({replaceStrings[6]:np.NaN}, regex=True)
feedbackComments['preference-reason'] = feedbackComments['preference-reason'].replace({replaceStrings[7]:np.NaN}, regex=True)
feedbackComments['preference-reason'] = feedbackComments['preference-reason'].replace({replaceStrings[8]:np.NaN}, regex=True)
feedbackComments['preference-reason'] = feedbackComments['preference-reason'].replace({replaceStrings[9]:np.NaN}, regex=True)
feedbackComments['preference-reason'] = feedbackComments['preference-reason'].replace({replaceStrings[10]:np.NaN}, regex=True)

# Chord Pors/Cons Replacements
feedbackComments['qualitative-questions-pros-cons-chord'] = feedbackComments['qualitative-questions-pros-cons-chord'].replace({replaceStrings[0]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-chord'] = feedbackComments['qualitative-questions-pros-cons-chord'].replace({replaceStrings[1]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-chord'] = feedbackComments['qualitative-questions-pros-cons-chord'].replace({replaceStrings[2]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-chord'] = feedbackComments['qualitative-questions-pros-cons-chord'].replace({replaceStrings[3]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-chord'] = feedbackComments['qualitative-questions-pros-cons-chord'].replace({replaceStrings[4]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-chord'] = feedbackComments['qualitative-questions-pros-cons-chord'].replace({replaceStrings[5]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-chord'] = feedbackComments['qualitative-questions-pros-cons-chord'].replace({replaceStrings[6]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-chord'] = feedbackComments['qualitative-questions-pros-cons-chord'].replace({replaceStrings[7]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-chord'] = feedbackComments['qualitative-questions-pros-cons-chord'].replace({replaceStrings[8]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-chord'] = feedbackComments['qualitative-questions-pros-cons-chord'].replace({replaceStrings[9]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-chord'] = feedbackComments['qualitative-questions-pros-cons-chord'].replace({replaceStrings[10]:np.NaN}, regex=True)

# Map Pors/Cons Replacements
feedbackComments['qualitative-questions-pros-cons-map'] = feedbackComments['qualitative-questions-pros-cons-map'].replace({replaceStrings[0]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-map'] = feedbackComments['qualitative-questions-pros-cons-map'].replace({replaceStrings[1]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-map'] = feedbackComments['qualitative-questions-pros-cons-map'].replace({replaceStrings[2]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-map'] = feedbackComments['qualitative-questions-pros-cons-map'].replace({replaceStrings[3]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-map'] = feedbackComments['qualitative-questions-pros-cons-map'].replace({replaceStrings[4]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-map'] = feedbackComments['qualitative-questions-pros-cons-map'].replace({replaceStrings[5]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-map'] = feedbackComments['qualitative-questions-pros-cons-map'].replace({replaceStrings[6]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-map'] = feedbackComments['qualitative-questions-pros-cons-map'].replace({replaceStrings[7]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-map'] = feedbackComments['qualitative-questions-pros-cons-map'].replace({replaceStrings[8]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-map'] = feedbackComments['qualitative-questions-pros-cons-map'].replace({replaceStrings[9]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-map'] = feedbackComments['qualitative-questions-pros-cons-map'].replace({replaceStrings[10]:np.NaN}, regex=True)

# List Pors/Cons Replacements
feedbackComments['qualitative-questions-pros-cons-list'] = feedbackComments['qualitative-questions-pros-cons-list'].replace({replaceStrings[0]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-list'] = feedbackComments['qualitative-questions-pros-cons-list'].replace({replaceStrings[1]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-list'] = feedbackComments['qualitative-questions-pros-cons-list'].replace({replaceStrings[2]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-list'] = feedbackComments['qualitative-questions-pros-cons-list'].replace({replaceStrings[3]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-list'] = feedbackComments['qualitative-questions-pros-cons-list'].replace({replaceStrings[4]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-list'] = feedbackComments['qualitative-questions-pros-cons-list'].replace({replaceStrings[5]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-list'] = feedbackComments['qualitative-questions-pros-cons-list'].replace({replaceStrings[6]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-list'] = feedbackComments['qualitative-questions-pros-cons-list'].replace({replaceStrings[7]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-list'] = feedbackComments['qualitative-questions-pros-cons-list'].replace({replaceStrings[8]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-list'] = feedbackComments['qualitative-questions-pros-cons-list'].replace({replaceStrings[9]:np.NaN}, regex=True)
feedbackComments['qualitative-questions-pros-cons-list'] = feedbackComments['qualitative-questions-pros-cons-list'].replace({replaceStrings[10]:np.NaN}, regex=True)

# comments Replacements
feedbackComments['comments'] = feedbackComments['comments'].replace({replaceStrings[0]:np.NaN}, regex=True)
feedbackComments['comments'] = feedbackComments['comments'].replace({replaceStrings[1]:np.NaN}, regex=True)
feedbackComments['comments'] = feedbackComments['comments'].replace({replaceStrings[2]:np.NaN}, regex=True)
feedbackComments['comments'] = feedbackComments['comments'].replace({replaceStrings[3]:np.NaN}, regex=True)
feedbackComments['comments'] = feedbackComments['comments'].replace({replaceStrings[4]:np.NaN}, regex=True)
feedbackComments['comments'] = feedbackComments['comments'].replace({replaceStrings[5]:np.NaN}, regex=True)
feedbackComments['comments'] = feedbackComments['comments'].replace({replaceStrings[6]:np.NaN}, regex=True)
feedbackComments['comments'] = feedbackComments['comments'].replace({replaceStrings[7]:np.NaN}, regex=True)
feedbackComments['comments'] = feedbackComments['comments'].replace({replaceStrings[8]:np.NaN}, regex=True)
feedbackComments['comments'] = feedbackComments['comments'].replace({replaceStrings[9]:np.NaN}, regex=True)
feedbackComments['comments'] = feedbackComments['comments'].replace({replaceStrings[10]:np.NaN}, regex=True)
pd.options.mode.chained_assignment = 'warn'  # default='warn'

cmolComments = feedbackComments['preference-reason'].count()
cComments = feedbackComments['qualitative-questions-pros-cons-chord'].count()
mComments = feedbackComments['qualitative-questions-pros-cons-map'].count()
lComments = feedbackComments['qualitative-questions-pros-cons-list'].count()
fcomComments = feedbackComments['comments'].count()

def nicePercentage(floatValue):
  percentage = np.round(floatValue/len(feedbackComments)*100, 2)
  return percentage

data = {
  'Feedback Art': ['cmolComments', 'cComments', 'mComments', 'lComments', 'fcomComments'],
  'Anzahl': [cmolComments, cComments, mComments, lComments, fcomComments],
  'Anteil': [nicePercentage(cmolComments), nicePercentage(cComments), nicePercentage(mComments), nicePercentage(lComments), nicePercentage(fcomComments)],
}

feedbackValues = pd.DataFrame(data)
feedbackValues

,Feedback Art,Anzahl,Anteil
0,cmolComments,191,94.09
1,cComments,184,90.64
2,mComments,188,92.61
3,lComments,180,88.67
4,fcomComments,73,35.96


### Quantitiy of Codes

In [40]:
qualitativeData=pd.read_csv('participants_codes.csv', sep=';')
# Code IDs mit Namen ausgeben

allCodesCounts = qualitativeData.groupby(['code_id', 'code.name', 'field_types.name'], dropna=False)['code_id'].count().reset_index(name='count').sort_values(by=['count', 'field_types.name'], ascending=False)
codeGroupsCounts = qualitativeData.groupby(['field_types.name'], dropna=False)['field_types.name'].count().reset_index(name='count').sort_values(by=['count', 'field_types.name'], ascending=False)
questionAllocationCounts = qualitativeData.groupby(['vizvar'], dropna=False)['vizvar'].count().reset_index(name='count').sort_values(by=['count'], ascending=False)
print(questionAllocationCounts)
print(len(questionAllocationCounts))
print("Sum Counts", allCodesCounts['count'].sum())

# codeGroupsCounts
# Label strings
chartLabel = f"Verteilung der Codes innerhalb der {len(codeGroupsCounts[['field_types.name']])} Gruppen"
categoryGroups = ["Chord spezifisch", "Map spezifisch", "Listen spezifisch", "GUI", "Personenbezogen"]

fig = make_subplots(rows=1, cols=1, 
    specs=[
        [{'type':'domain'}],
    ],
    # subplot_titles="Chord oder Map favorisiert"
)

fig.add_trace(go.Pie(labels=categoryGroups, values=codeGroupsCounts['count'], sort=False, direction='clockwise', marker_colors=urCdColors),
              1, 1)

fig.update_traces(textinfo='percent+label+value')

fig.update_layout(
    title={
        'text': chartLabel,
        'y':0.95,
        # 'x':0,
        'xanchor': 'left',
        'yanchor': 'top',
        'font_size': 18,
    },
    # yaxis_title=yLabel,
    width=850,
    height=950,
    #legend_title="Legend Title",
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=24,
        # color="RebeccaPurple"
    )
)
fig.update(layout_showlegend=True)
# codeGroupsCounts
fig.show()

# allCodesCounts
allCodesCounts['field_types.name'] = allCodesCounts['field_types.name'].replace({'GUI ':'GUI'}, regex=True)
allCodesCounts['label'] = allCodesCounts['field_types.name'] +  ": " + allCodesCounts['code.name']
allCodesCounts['textValue'] = np.round(allCodesCounts['count']/len(qualitativeData)*100, 2).astype(str) + " % (" + allCodesCounts['count'].astype(str) + ")"

# Label strings
chartLabel = f'Verteilung nach Codes von {len(qualitativeData[["code.name"]]):,}'.replace(",",".") + ' Codierungen'

fig = px.bar(allCodesCounts, y='label', x='count', text=allCodesCounts['textValue'], 
    color_discrete_sequence =urCdColors*3, 
    orientation='h', 
    # category_orders={
    #     'code.name"': allCodesCounts['code.name'].tolist()
    # }
)

fig.update_traces(textposition="outside", cliponaxis=False)
fig.update_layout(
    title=chartLabel,
    yaxis_title='Code',
    xaxis_title='Häufigkeit des Codes',
    # width=1080,
    height=1920,
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=21,
        # color="RebeccaPurple"
    )
)
fig.update_layout(yaxis=dict(autorange="reversed"))
fig.update_yaxes(ticksuffix = "  ")
fig.show()

# allCodesProParticipants
allCodesCounts['textParticipantValue'] = np.round(allCodesCounts['count']/194*100, 2).astype(str) + " % (" + allCodesCounts['count'].astype(str) + ")"

# Label strings
chartLabel = f'Verteilung nach Codes bei 194 Teilnehmern'

fig = px.bar(allCodesCounts, y='label', x='count', text=allCodesCounts['textParticipantValue'], 
    color_discrete_sequence =urCdColors[1:]*3,
    orientation='h', 
    # category_orders={
    #     'code.name"': allCodesCounts['code.name'].tolist()
    # }
)

fig.update_traces(textposition="outside", cliponaxis=False)
fig.update_layout(
    title=chartLabel,
    yaxis_title='Code',
    xaxis_title='Häufigkeit des Codes',
    # width=1080,
    height=1920,
    font=dict(
        family="Frutiger Next LT W1G, Gill Sans, sans-serif",
        size=21,
        # color="RebeccaPurple"
    )
)
fig.update_layout(yaxis=dict(autorange="reversed"))
fig.update_yaxes(ticksuffix = "  ")
fig.show()


             vizvar  count
0             chord    439
4               map    376
1  chordMapOverList    332
3              list    294
2      finalComment     25
5
Sum Counts 1466
